In [2]:
import pandas as pd 
import numpy as np 

df = pd.read_csv( "emergencias_julio_2021.csv",
    sep=";",              # muy importante para este archivo
    encoding="utf-8"      # casi siempre funciona bien en español
)
#encoding sirve para los acentos y caracteres especiales
#"utf-8" es un estandar muy usado 


In [3]:
#diagnostico
print(df.shape)
df.info()

(314507, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314507 entries, 0 to 314506
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Fecha          314507 non-null  object
 1   provincia      314507 non-null  object
 2   Canton         314507 non-null  object
 3   Cod_Parroquia  314507 non-null  int64 
 4   Parroquia      314507 non-null  object
 5   Servicio       314507 non-null  object
 6   Subtipo        314507 non-null  object
dtypes: int64(1), object(6)
memory usage: 16.8+ MB


In [4]:
import sys
!{sys.executable} -m pip install unidecode


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from unidecode import unidecode
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
def norm_txt(s):
    if pd.isna(s):
        return None   # si ya es nulo, retorna None
    
    s = str(s).strip()
    s = " ".join(s.split())   # colapsa espacios múltiples
    s = unidecode(s).lower()  # sin acentos y minúsculas
    
    # Si la provincia es inválida → retorna None
    if s in ['0', 'zona no delimitada']:
        return None
    
    return s   # valor normalizado


In [7]:
df0 = df.copy()
df0

Text_columns = ['provincia' , 'Canton' , 'Parroquia', 'Servicio', 'Subtipo']
for col in Text_columns:
    if col in df0.columns :
        df0[col] = df0[col].apply(norm_txt)
        

if 'Cod_Parroquia' in df0.columns:
    df0['Cod_Parroquia'] = (
        df0['Cod_Parroquia']
        .astype(str)
        .str.replace(r'\.0$', '', regex=True)  # elimina .0 si vino de Excel
        .str.strip()
    )

print(df0['Cod_Parroquia'].dtypes)
print(df0['Cod_Parroquia'].head())

object
0    90150
1    90650
2    90950
3    92050
4    90150
Name: Cod_Parroquia, dtype: object


In [8]:
df0

,Fecha,provincia,Canton,Cod_Parroquia,Parroquia,Servicio,Subtipo
0,2021-07-01,guayas,guayaquil,90150,"guayaquil, cabecera cantonal y capital provincial",seguridad ciudadana,libadores
1,2021-07-01,guayas,daule,90650,"daule, cabecera cantonal",seguridad ciudadana,escandalo en espacio privado
2,2021-07-01,guayas,el triunfo,90950,"el triunfo, cabecera cantonal",seguridad ciudadana,escandalo en espacio privado
3,2021-07-01,guayas,san jacinto de yaguachi,92050,"san jacinto de yaguachi, cabecera cantonal",seguridad ciudadana,patrullaje policial en el sector solicitado
4,2021-07-01,guayas,guayaquil,90150,"guayaquil, cabecera cantonal y capital provincial",seguridad ciudadana,presencia policial
...,...,...,...,...,...,...,...
314502,2021-07-31,tungurahua,santiago de pillaro,180852,emilio maria teran (rumipamba),seguridad ciudadana,escandalo en espacio privado
314503,2021-07-31,tungurahua,ambato,180150,"ambato, cabecera cantonal y capital provincial",seguridad ciudadana,libadores
314504,2021-07-31,pastaza,pastaza,160150,"puyo, cabecera cantonal y capital provincial",seguridad ciudadana,libadores
314505,2021-07-31,tungurahua,ambato,180150,"ambato, cabecera cantonal y capital provincial",seguridad ciudadana,ficha de datos informacion


In [9]:
df0['provincia'].isna().sum()
eventos_none = df0[df0['provincia'].isna()]

print("===== EVENTOS CON provincia = None =====")
print(eventos_none)
print(f"\nTotal de eventos con None: {len(eventos_none)}")
print(df0.shape)




===== EVENTOS CON provincia = None =====
            Fecha provincia       Canton Cod_Parroquia    Parroquia  \
16675  2021-07-17      None  el piedrero        900451  el piedrero   
19167  2021-07-18      None  el piedrero        900451  el piedrero   
25202  2021-07-24      None  el piedrero        900451  el piedrero   
27721  2021-07-26      None  el piedrero        900451  el piedrero   
64891  2021-07-02      None         None             0         None   
...           ...       ...          ...           ...          ...   
248922 2021-07-24      None         None             0         None   
262456 2021-07-14      None         None             0         None   
262502 2021-07-16      None         None             0         None   
262697 2021-07-23      None         None             0         None   
262932 2021-07-31      None         None             0         None   

                     Servicio                       Subtipo  
16675       gestion sanitaria               

In [10]:
# Contar antes de eliminar
antes = df0.shape[0]

# Eliminar filas donde provincia es None/NaN/pd.NA
df0 = df0[df0['provincia'].notna()].copy()

# Contar después
despues = df0.shape[0]

print(f"Filas antes: {antes}")
print(f"Filas después: {despues}")
print(f"Filas eliminadas: {antes - despues}")


Filas antes: 314507
Filas después: 314417
Filas eliminadas: 90


In [11]:
df0.shape

(314417, 7)

In [12]:
# ----- 1) Define la llave del evento único -----
keys = [
    'Fecha',
    'provincia_limpia',
    'Canton',
    'Parroquia',
    'Cod_Parroquia',
    'Servicio',
    'Subtipo'
]

# Filtrar solo las que existan en tu DataFrame
keys = [c for c in keys if c in df0.columns]


# ----- 2) Identificar duplicados según esa llave -----
# df0.duplicated(subset=keys, keep=False) marca TODOS los duplicados (incluye el primero)
mask_dups = df0.duplicated(subset=keys, keep=False)

# Extraer los eventos duplicados
duplicados = df0.loc[mask_dups].sort_values(keys)

# Mostrar resumen
print(f"Total de filas duplicadas usando la llave: {len(duplicados)}")

# Mostrar los duplicados
print("\n===== EVENTOS DUPLICADOS =====")
print(duplicados)


Total de filas duplicadas usando la llave: 244790

===== EVENTOS DUPLICADOS =====
           Fecha   provincia     Canton Cod_Parroquia  \
33179 2021-07-01  chimborazo     alausi         60250   
33188 2021-07-01  chimborazo     alausi         60250   
33206 2021-07-01  chimborazo     alausi         60250   
33223 2021-07-01  chimborazo     alausi         60250   
33311 2021-07-01  chimborazo     alausi         60250   
...          ...         ...        ...           ...   
75977 2021-07-31        loja  zapotillo        111350   
42078 2021-07-31      el oro     zaruma         71350   
43820 2021-07-31      el oro     zaruma         71350   
43940 2021-07-31      el oro     zaruma         71350   
43941 2021-07-31      el oro     zaruma         71350   

                          Parroquia             Servicio  \
33179     alausi, cabecera cantonal  seguridad ciudadana   
33188     alausi, cabecera cantonal  seguridad ciudadana   
33206     alausi, cabecera cantonal  seguridad ciudad

In [13]:
PROVINCIAS_VALIDAS = {
    '01':'AZUAY','02':'BOLIVAR','03':'CANAR','04':'CARCHI','05':'COTOPAXI','06':'CHIMBORAZO',
    '07':'EL ORO','08':'ESMERALDAS','09':'GUAYAS','10':'IMBABURA','11':'LOJA','12':'LOS RIOS',
    '13':'MANABI','14':'MORONA SANTIAGO','15':'NAPO','16':'PASTAZA','17':'PICHINCHA',
    '18':'TUNGURAHUA','19':'ZAMORA CHINCHIPE','20':'GALAPAGOS','21':'SUCUMBIOS',
    '22':'ORELLANA','23':'SANTO DOMINGO DE LOS TSACHILAS','24':'SANTA ELENA'
}
VALIDAS_SET = set(PROVINCIAS_VALIDAS.values())

In [14]:
df0.columns

Index(['Fecha', 'provincia', 'Canton', 'Cod_Parroquia', 'Parroquia',
       'Servicio', 'Subtipo'],
      dtype='object')

In [15]:
#validaciones de contenido:
# Provincias inválidas remanentes
restantes = df0.loc[df0['provincia'].isin(VALIDAS_SET) & df0['provincia'].notna()]
print(len(restantes))  # debería ser 0 si no usas 'SIN PROVINCIA'

# Nulos por columna clave
print(df0[['Fecha', 'provincia', 'Canton', 'Cod_Parroquia', 'Parroquia',
       'Servicio', 'Subtipo']].isna().sum())


0
Fecha            0
provincia        0
Canton           0
Cod_Parroquia    0
Parroquia        0
Servicio         0
Subtipo          0
dtype: int64


In [16]:
df0

,Fecha,provincia,Canton,Cod_Parroquia,Parroquia,Servicio,Subtipo
0,2021-07-01,guayas,guayaquil,90150,"guayaquil, cabecera cantonal y capital provincial",seguridad ciudadana,libadores
1,2021-07-01,guayas,daule,90650,"daule, cabecera cantonal",seguridad ciudadana,escandalo en espacio privado
2,2021-07-01,guayas,el triunfo,90950,"el triunfo, cabecera cantonal",seguridad ciudadana,escandalo en espacio privado
3,2021-07-01,guayas,san jacinto de yaguachi,92050,"san jacinto de yaguachi, cabecera cantonal",seguridad ciudadana,patrullaje policial en el sector solicitado
4,2021-07-01,guayas,guayaquil,90150,"guayaquil, cabecera cantonal y capital provincial",seguridad ciudadana,presencia policial
...,...,...,...,...,...,...,...
314502,2021-07-31,tungurahua,santiago de pillaro,180852,emilio maria teran (rumipamba),seguridad ciudadana,escandalo en espacio privado
314503,2021-07-31,tungurahua,ambato,180150,"ambato, cabecera cantonal y capital provincial",seguridad ciudadana,libadores
314504,2021-07-31,pastaza,pastaza,160150,"puyo, cabecera cantonal y capital provincial",seguridad ciudadana,libadores
314505,2021-07-31,tungurahua,ambato,180150,"ambato, cabecera cantonal y capital provincial",seguridad ciudadana,ficha de datos informacion


# Parroquias INEC

In [17]:
dataI = pd.read_excel("CODIFICACIÓN_2021.xlsx", header=0 , skiprows=1)
dataI


,Unnamed: 0,DPA_PROVIN,DPA_DESPRO,DPA_CANTON,DPA_DESCAN,DPA_PARROQ,DPA_DESPAR,Unnamed: 7,DPA_PARROQ.1,DPA_DESPAR.1,Revisión,Revisión .1
0,NaN,1,AZUAY,101,CUENCA,10150,CUENCA,NaN,10150,CUENCA,True,True
1,NaN,1,AZUAY,101,CUENCA,10151,BAÑOS,NaN,10151,BAÑOS,True,True
2,NaN,1,AZUAY,101,CUENCA,10152,CUMBE,NaN,10152,CUMBE,True,True
3,NaN,1,AZUAY,101,CUENCA,10153,CHAUCHA,NaN,10153,CHAUCHA,True,True
4,NaN,1,AZUAY,101,CUENCA,10154,CHECA,NaN,10154,CHECA,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...
1037,NaN,24,SANTA ELENA,2401,SANTA ELENA,240156,SAN JOSÉ DE ANCÓN,NaN,240156,SAN JOSÉ DE ANCÓN,True,True
1038,NaN,24,SANTA ELENA,2402,LA LIBERTAD,240250,LA LIBERTAD,NaN,240250,LA LIBERTAD,True,True
1039,NaN,24,SANTA ELENA,2403,SALINAS,240350,SALINAS,NaN,240350,SALINAS,True,True
1040,NaN,24,SANTA ELENA,2403,SALINAS,240351,ANCONCITO,NaN,240351,ANCONCITO,True,True


In [18]:
dataI = dataI.dropna(axis=1,how="any")
dataI

,DPA_PROVIN,DPA_DESPRO,DPA_CANTON,DPA_DESCAN,DPA_PARROQ,DPA_DESPAR,DPA_PARROQ.1,DPA_DESPAR.1,Revisión,Revisión .1
0,1,AZUAY,101,CUENCA,10150,CUENCA,10150,CUENCA,True,True
1,1,AZUAY,101,CUENCA,10151,BAÑOS,10151,BAÑOS,True,True
2,1,AZUAY,101,CUENCA,10152,CUMBE,10152,CUMBE,True,True
3,1,AZUAY,101,CUENCA,10153,CHAUCHA,10153,CHAUCHA,True,True
4,1,AZUAY,101,CUENCA,10154,CHECA,10154,CHECA,True,True
...,...,...,...,...,...,...,...,...,...,...
1037,24,SANTA ELENA,2401,SANTA ELENA,240156,SAN JOSÉ DE ANCÓN,240156,SAN JOSÉ DE ANCÓN,True,True
1038,24,SANTA ELENA,2402,LA LIBERTAD,240250,LA LIBERTAD,240250,LA LIBERTAD,True,True
1039,24,SANTA ELENA,2403,SALINAS,240350,SALINAS,240350,SALINAS,True,True
1040,24,SANTA ELENA,2403,SALINAS,240351,ANCONCITO,240351,ANCONCITO,True,True


In [19]:
dataI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1042 entries, 0 to 1041
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   DPA_PROVIN    1042 non-null   int64 
 1   DPA_DESPRO    1042 non-null   object
 2   DPA_CANTON    1042 non-null   int64 
 3   DPA_DESCAN    1042 non-null   object
 4   DPA_PARROQ    1042 non-null   int64 
 5   DPA_DESPAR    1042 non-null   object
 6   DPA_PARROQ.1  1042 non-null   int64 
 7   DPA_DESPAR.1  1042 non-null   object
 8   Revisión      1042 non-null   bool  
 9   Revisión .1   1042 non-null   bool  
dtypes: bool(2), int64(4), object(4)
memory usage: 67.3+ KB


In [20]:
cod_parroquia_inec = dataI['DPA_PARROQ']
cod_parroquia_inec

0        10150
1        10151
2        10152
3        10153
4        10154
         ...  
1037    240156
1038    240250
1039    240350
1040    240351
1041    240352
Name: DPA_PARROQ, Length: 1042, dtype: int64

In [21]:
# Convertir a numéricos y limpiar
cod_p = pd.to_numeric(df0['Cod_Parroquia'], errors='coerce').dropna().astype(int)
cod_inec = pd.to_numeric(cod_parroquia_inec, errors='coerce').dropna().astype(int)

# Encontrar cuáles códigos de tu data NO están en INEC
faltantes = cod_p[~cod_p.isin(cod_inec)]

print(f"Total de códigos en tu data: {len(cod_p)}")
print(f"Total de códigos en INEC: {len(cod_inec)}")
print(f"\n{'='*50}")

if len(faltantes) > 0:
    print(f"✗ Códigos que NO existen en INEC: {len(faltantes.unique())}")
    print(f"\nDetalle:")
    for codigo in sorted(faltantes.unique()):
        count = (cod_p == codigo).sum()
        print(f"  Código {codigo}: aparece {count} veces")
else:
    print(f"✓ TODOS los códigos existen en INEC")

faltantes


Total de códigos en tu data: 314417
Total de códigos en INEC: 1042

✗ Códigos que NO existen en INEC: 1

Detalle:
  Código 11551: aparece 11 veces


43403     11551
86150     11551
87671     11551
90126     11551
91119     11551
91805     11551
92287     11551
100100    11551
100176    11551
100194    11551
102755    11551
Name: Cod_Parroquia, dtype: int64

In [22]:
faltantes_set = set(cod_p) - set(cod_inec)
faltantes = sorted(faltantes_set)
print(f"Códigos faltantes: {faltantes}")

Códigos faltantes: [11551]


# Código más modular

In [23]:
df1 = df.copy()

In [24]:
df1

,Fecha,provincia,Canton,Cod_Parroquia,Parroquia,Servicio,Subtipo
0,2021-07-01,GUAYAS,GUAYAQUIL,90150,"GUAYAQUIL, CABECERA CANTONAL Y CAPITAL PROVINCIAL",Seguridad Ciudadana,Libadores
1,2021-07-01,GUAYAS,DAULE,90650,"DAULE, CABECERA CANTONAL",Seguridad Ciudadana,Escándalo en espacio privado
2,2021-07-01,GUAYAS,EL TRIUNFO,90950,"EL TRIUNFO, CABECERA CANTONAL",Seguridad Ciudadana,Escándalo en espacio privado
3,2021-07-01,GUAYAS,SAN JACINTO DE YAGUACHI,92050,"SAN JACINTO DE YAGUACHI, CABECERA CANTONAL",Seguridad Ciudadana,Patrullaje policial en el sector solicitado
4,2021-07-01,GUAYAS,GUAYAQUIL,90150,"GUAYAQUIL, CABECERA CANTONAL Y CAPITAL PROVINCIAL",Seguridad Ciudadana,Presencia policial
...,...,...,...,...,...,...,...
314502,2021-07-31,TUNGURAHUA,SANTIAGO DE PILLARO,180852,EMILIO MARÍA TERÁN (RUMIPAMBA),Seguridad Ciudadana,Escándalo en espacio privado
314503,2021-07-31,TUNGURAHUA,AMBATO,180150,"AMBATO, CABECERA CANTONAL Y CAPITAL PROVINCIAL",Seguridad Ciudadana,Libadores
314504,2021-07-31,PASTAZA,PASTAZA,160150,"PUYO, CABECERA CANTONAL Y CAPITAL PROVINCIAL",Seguridad Ciudadana,Libadores
314505,2021-07-31,TUNGURAHUA,AMBATO,180150,"AMBATO, CABECERA CANTONAL Y CAPITAL PROVINCIAL",Seguridad Ciudadana,Ficha de datos información


In [25]:
import pandas as pd
from unidecode import unidecode


# ============================================================
# 1. NORMALIZACIÓN DE TEXTO
# ============================================================

def norm_nombre(s):
    """
    Normaliza nombres de provincia/cantón/parroquia:
    - strip espacios
    - colapsa espacios múltiples
    - quita acentos
    - pasa a minúsculas
    """
    if pd.isna(s):
        return None
    s = str(s).strip()
    s = " ".join(s.split())
    s = unidecode(s).lower()
    return s


def norm_provincia(s):
    """
    Normaliza provincia y convierte a None si es '0' o 'zona no delimitada'.
    """
    s_norm = norm_nombre(s)
    if s_norm in ['0', 'zona no delimitada']:
        return None
    return s_norm


# ============================================================
# 2. CARGAR DATASET DE EMERGENCIAS
# ============================================================

def load_emergencias(path_csv: str) -> pd.DataFrame:
    """
    Lee el CSV de emergencias (julio) con los parámetros correctos.
    """
    df = pd.read_csv(path_csv, sep=";", encoding="utf-8")
    df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
    return df


# ============================================================
# 3. LIMPIEZA DE EMERGENCIAS
# ============================================================

def clean_emergencias(df: pd.DataFrame) -> pd.DataFrame:
    """
    Limpia el dataset de emergencias:
    - Normaliza texto en provincia, cantón, parroquia, servicio, subtipo
    - Marca provincias inválidas ('0', 'zona no delimitada') como None
    - Limpia Cod_Parroquia y la deja como string (6 dígitos cuando aplique)
    - Crea columnas prov_norm, canton_norm, parr_norm
    - Elimina filas sin provincia (None)
    """
    df0 = df.copy()

    # Normalizar columnas de texto
    text_cols = ['provincia', 'Canton', 'Parroquia', 'Servicio', 'Subtipo']
    for col in text_cols:
        if col not in df0.columns:
            continue
        if col == 'provincia':
            df0[col] = df0[col].apply(norm_provincia)
        else:
            df0[col] = df0[col].apply(norm_nombre)
            
    if 'Cod_Parroquia' in df0.columns:
        df0['Cod_Parroquia'] = (
            df0['Cod_Parroquia']
            .astype(str)
            .str.replace(r'\.0$', '', regex=True)
            .str.strip()
        )

    df0['Cod_Parroquia'] = df0['Cod_Parroquia'].apply(
        lambda x: x.zfill(6) if x.isdigit() and len(x) <= 6 else x
    )

    

    # Columnas normalizadas explícitas (para el match con INEC)
    df0['prov_norm']   = df0['provincia'].apply(norm_nombre)
    df0['canton_norm'] = df0['Canton'].apply(norm_nombre)
    df0['parr_norm']   = df0['Parroquia'].apply(norm_nombre)

    # Eliminar filas sin provincia (None/NaN) porque no se pueden georreferenciar
    antes = len(df0)
    df0 = df0[df0['provincia'].notna()].copy()
    despues = len(df0)
    print(f"[clean_emergencias] Filas eliminadas por provincia None: {antes - despues}")

    return df0


# ============================================================
# 4. CARGAR Y PREPARAR CODIFICACIÓN INEC 2021
# ============================================================

def load_inec_codificacion(dataI: pd.DataFrame) -> pd.DataFrame:
    """
    Lee el archivo CODIFICACIÓN_2021 del INEC y prepara un DataFrame de referencia
    con:
    - DPA_PARROQ (código parroquial de 6 dígitos)
    - Nombres oficiales de provincia, cantón, parroquia
    - Versiones normalizadas para el match (prov_norm, canton_norm, parr_norm)
    """
    cols = ['DPA_PROVIN', 'DPA_DESPRO',
            'DPA_CANTON', 'DPA_DESCAN',
            'DPA_PARROQ', 'DPA_DESPAR']
    inec_ref = dataI[cols].copy()
    
    # Normalizar nombres
    inec_ref['prov_norm']   = inec_ref['DPA_DESPRO'].apply(norm_nombre)
    inec_ref['canton_norm'] = inec_ref['DPA_DESCAN'].apply(norm_nombre)
    inec_ref['parr_norm']   = inec_ref['DPA_DESPAR'].apply(norm_nombre)

    return inec_ref


# ============================================================
# 5. MAPEAR PARROQUIAS A CÓDIGO INEC
# ============================================================

def mapear_parroquias_inec(df_emerg: pd.DataFrame, inec_ref: pd.DataFrame) -> pd.DataFrame:
    """
    Mapea el código de parroquia de df_emerg contra el catálogo INEC.
    
    Si Cod_Parroquia coincide con DPA_PARROQ -> asigna el código.
    Si NO coincide -> coloca NaN.
    """
    # Nos aseguramos que los códigos estén limpios y comparables
    df = df_emerg.copy()
    df['Cod_Parroquia'] = (
        df['Cod_Parroquia']
        .astype(str)
        .str.replace(r'\.0$', '', regex=True)
        .str.strip()
        .str.zfill(6)
    )
    
    inec = inec_ref.copy()
    inec['DPA_PARROQ'] = (
        inec['DPA_PARROQ']
        .astype(str)
        .str.replace(r'\.0$', '', regex=True)
        .str.strip()
        .str.zfill(6)
    )

    # Merge SOLO por código parroquial
    df_geo = df.merge(
        inec[['DPA_PARROQ']],   # solo necesitamos el código oficial
        left_on='Cod_Parroquia',
        right_on='DPA_PARROQ',
        how='left'
    )

    return df_geo



# ============================================================
# 6. REPORTE DE GEOREFERENCIACIÓN
# ============================================================

def reporte_geocodificacion(df_geo: pd.DataFrame) -> None:
    """
    Imprime un resumen de qué tan bien se mapeó a INEC:
    - % de filas con código parroquial asignado
    - ejemplos de parroquias sin match
    """
    total = len(df_geo)
    con_codigo = df_geo['DPA_PARROQ'].notna().sum()
    sin_codigo = total - con_codigo

    print("=== REPORTE GEOREFERENCIACIÓN ===")
    print(f"Filas totales: {total}")
    print(f"Con código INEC (DPA_PARROQ): {con_codigo} ({con_codigo/total*100:.2f}%)")
    print(f"Sin código INEC: {sin_codigo} ({sin_codigo/total*100:.2f}%)")

    if sin_codigo > 0:
        print("\nEjemplos de parroquias SIN match (provincia / cantón / parroquia):")
        ejemplos = (
            df_geo[df_geo['DPA_PARROQ'].isna()]
            [['provincia', 'Canton', 'Parroquia']]
            .drop_duplicates()
            .head(300)
        )
        print(ejemplos)


# ============================================================
# 7. PIPELINE COMPLETO
# ============================================================

def pipeline_georreferenciacion(path_emerg: str, dataI: pd.DataFrame) -> pd.DataFrame:
    """
    Ejecuta todo el flujo:
    1) Carga emergencias
    2) Limpia texto, provincias y Cod_Parroquia
    3) Carga catálogo INEC
    4) Mapea parroquias y agrega DPA_PARROQ
    5) Imprime reporte
    Devuelve df_geo (listo para unir con shapefile).
    """
    print("1) Cargando emergencias...")
    df = load_emergencias(path_emerg)

    print("2) Limpiando emergencias...")
    df_clean = clean_emergencias(df)

    print("3) Cargando codificación INEC...")
    inec_ref = load_inec_codificacion(dataI)

    print("4) Mapeando parroquias a INEC...")
    df_geo = mapear_parroquias_inec(df_clean, inec_ref)

    print("5) Reporte de georreferenciación:")
    reporte_geocodificacion(df_geo)

    return df_geo


# ============================================================
# 8. EJEMPLO DE USO
# ============================================================

# Descomenta esto y ajusta las rutas en tu notebook/script:
df_geo = pipeline_georreferenciacion(
     path_emerg="emergencias_julio_2021.csv",
     dataI=dataI
)
#df_geo.to_csv("emergencias_julio_2021_georreferenciado.csv", index=False)
df_geo




1) Cargando emergencias...
2) Limpiando emergencias...
[clean_emergencias] Filas eliminadas por provincia None: 90
3) Cargando codificación INEC...
4) Mapeando parroquias a INEC...
5) Reporte de georreferenciación:
=== REPORTE GEOREFERENCIACIÓN ===
Filas totales: 314417
Con código INEC (DPA_PARROQ): 314406 (100.00%)
Sin código INEC: 11 (0.00%)

Ejemplos de parroquias SIN match (provincia / cantón / parroquia):
      provincia                 Canton            Parroquia
43399     azuay  camilo ponce enriquez  el carmen de pijili


,Fecha,provincia,Canton,Cod_Parroquia,Parroquia,Servicio,Subtipo,prov_norm,canton_norm,parr_norm,DPA_PARROQ
0,2021-07-01,guayas,guayaquil,090150,"guayaquil, cabecera cantonal y capital provincial",seguridad ciudadana,libadores,guayas,guayaquil,"guayaquil, cabecera cantonal y capital provincial",090150
1,2021-07-01,guayas,daule,090650,"daule, cabecera cantonal",seguridad ciudadana,escandalo en espacio privado,guayas,daule,"daule, cabecera cantonal",090650
2,2021-07-01,guayas,el triunfo,090950,"el triunfo, cabecera cantonal",seguridad ciudadana,escandalo en espacio privado,guayas,el triunfo,"el triunfo, cabecera cantonal",090950
3,2021-07-01,guayas,san jacinto de yaguachi,092050,"san jacinto de yaguachi, cabecera cantonal",seguridad ciudadana,patrullaje policial en el sector solicitado,guayas,san jacinto de yaguachi,"san jacinto de yaguachi, cabecera cantonal",092050
4,2021-07-01,guayas,guayaquil,090150,"guayaquil, cabecera cantonal y capital provincial",seguridad ciudadana,presencia policial,guayas,guayaquil,"guayaquil, cabecera cantonal y capital provincial",090150
...,...,...,...,...,...,...,...,...,...,...,...
314412,2021-07-31,tungurahua,santiago de pillaro,180852,emilio maria teran (rumipamba),seguridad ciudadana,escandalo en espacio privado,tungurahua,santiago de pillaro,emilio maria teran (rumipamba),180852
314413,2021-07-31,tungurahua,ambato,180150,"ambato, cabecera cantonal y capital provincial",seguridad ciudadana,libadores,tungurahua,ambato,"ambato, cabecera cantonal y capital provincial",180150
314414,2021-07-31,pastaza,pastaza,160150,"puyo, cabecera cantonal y capital provincial",seguridad ciudadana,libadores,pastaza,pastaza,"puyo, cabecera cantonal y capital provincial",160150
314415,2021-07-31,tungurahua,ambato,180150,"ambato, cabecera cantonal y capital provincial",seguridad ciudadana,ficha de datos informacion,tungurahua,ambato,"ambato, cabecera cantonal y capital provincial",180150


In [26]:
def reporte_geocodificacion(df_geo: pd.DataFrame) -> pd.DataFrame:
    """
    Imprime un resumen de georreferenciación y retorna
    TODOS los eventos que NO tienen código INEC (DPA_PARROQ = NaN).
    """
    total = len(df_geo)
    con_codigo = df_geo['DPA_PARROQ'].notna().sum()
    sin_codigo = total - con_codigo

    print("=== REPORTE GEOREFERENCIACIÓN ===")
    print(f"Filas totales: {total}")
    print(f"Con código INEC (DPA_PARROQ): {con_codigo} ({con_codigo/total*100:.2f}%)")
    print(f"Sin código INEC: {sin_codigo} ({sin_codigo/total*100:.2f}%)")

    # Filtrar TODOS los eventos sin código INEC
    eventos_sin_codigo = df_geo[df_geo['DPA_PARROQ'].isna()].copy()

    print("\n===== EVENTOS SIN CÓDIGO INEC (DPA_PARROQ NaN) =====")
    print(eventos_sin_codigo)  # ojo: puede ser grande

    return eventos_sin_codigo



In [27]:
eventos_sin_codigo = reporte_geocodificacion(df_geo)

=== REPORTE GEOREFERENCIACIÓN ===
Filas totales: 314417
Con código INEC (DPA_PARROQ): 314406 (100.00%)
Sin código INEC: 11 (0.00%)

===== EVENTOS SIN CÓDIGO INEC (DPA_PARROQ NaN) =====
            Fecha provincia                 Canton Cod_Parroquia  \
43399  2021-07-23     azuay  camilo ponce enriquez        011551   
86139  2021-07-13     azuay  camilo ponce enriquez        011551   
87659  2021-07-01     azuay  camilo ponce enriquez        011551   
90111  2021-07-07     azuay  camilo ponce enriquez        011551   
91103  2021-07-09     azuay  camilo ponce enriquez        011551   
91788  2021-07-10     azuay  camilo ponce enriquez        011551   
92270  2021-07-11     azuay  camilo ponce enriquez        011551   
100071 2021-07-27     azuay  camilo ponce enriquez        011551   
100147 2021-07-27     azuay  camilo ponce enriquez        011551   
100165 2021-07-27     azuay  camilo ponce enriquez        011551   
102722 2021-07-03     azuay  camilo ponce enriquez        011551   

In [28]:
# Exporta a Excel sin índice
output_path = "df_geo_export.xlsx"
df_geo.to_excel(output_path, index=False)
print(f"Archivo guardado en: {output_path}")


Archivo guardado en: df_geo_export.xlsx


In [30]:
import pandas as pd
from unidecode import unidecode
import glob
import os


# ============================================================
# 1. NORMALIZACIÓN DE TEXTO
# ============================================================

def norm_nombre(s):
    """
    Normaliza nombres de provincia/cantón/parroquia:
    - strip espacios
    - colapsa espacios múltiples
    - quita acentos
    - pasa a minúsculas
    """
    if pd.isna(s):
        return None
    s = str(s).strip()
    s = " ".join(s.split())
    s = unidecode(s).lower()
    return s


def norm_provincia(s):
    """
    Normaliza provincia y convierte a None si es '0' o 'zona no delimitada'.
    """
    s_norm = norm_nombre(s)
    if s_norm in ['0', 'zona no delimitada']:
        return None
    return s_norm


# ============================================================
# 2. CARGAR DATASET DE EMERGENCIAS
# ============================================================

def load_emergencias(path_csv: str) -> pd.DataFrame:
    """
    Lee el CSV de emergencias con los parámetros correctos.
    """
    df = pd.read_csv(path_csv, sep=";", encoding="utf-8")
    df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
    return df


# ============================================================
# 3. LIMPIEZA DE EMERGENCIAS
# ============================================================

def clean_emergencias(df: pd.DataFrame) -> pd.DataFrame:
    """
    Limpia el dataset de emergencias:
    - Normaliza texto en provincia, cantón, parroquia, servicio, subtipo
    - Marca provincias inválidas ('0', 'zona no delimitada') como None
    - Limpia Cod_Parroquia y la deja como string (6 dígitos cuando aplique)
    - Crea columnas prov_norm, canton_norm, parr_norm
    - Elimina filas sin provincia (None)
    """
    df0 = df.copy()

    # Normalizar columnas de texto
    text_cols = ['provincia', 'Canton', 'Parroquia', 'Servicio', 'Subtipo']
    for col in text_cols:
        if col not in df0.columns:
            continue
        if col == 'provincia':
            df0[col] = df0[col].apply(norm_provincia)
        else:
            df0[col] = df0[col].apply(norm_nombre)
            
    if 'Cod_Parroquia' in df0.columns:
        df0['Cod_Parroquia'] = (
            df0['Cod_Parroquia']
            .astype(str)
            .str.replace(r'\.0$', '', regex=True)
            .str.strip()
        )

    df0['Cod_Parroquia'] = df0['Cod_Parroquia'].apply(
        lambda x: x.zfill(6) if x.isdigit() and len(x) <= 6 else x
    )

    

    # Columnas normalizadas explícitas (para el match con INEC)
    df0['prov_norm']   = df0['provincia'].apply(norm_nombre)
    df0['canton_norm'] = df0['Canton'].apply(norm_nombre)
    df0['parr_norm']   = df0['Parroquia'].apply(norm_nombre)

    # Eliminar filas sin provincia (None/NaN) porque no se pueden georreferenciar
    antes = len(df0)
    df0 = df0[df0['provincia'].notna()].copy()
    despues = len(df0)
    print(f"[clean_emergencias] Filas eliminadas por provincia None: {antes - despues}")

    return df0


# ============================================================
# 4. CARGAR Y PREPARAR CODIFICACIÓN INEC 2021
# ============================================================

def load_inec_codificacion(dataI: pd.DataFrame) -> pd.DataFrame:
    """
    Lee el archivo CODIFICACIÓN_2021 del INEC y prepara un DataFrame de referencia
    con:
    - DPA_PARROQ (código parroquial de 6 dígitos)
    - Nombres oficiales de provincia, cantón, parroquia
    - Versiones normalizadas para el match (prov_norm, canton_norm, parr_norm)
    """
    cols = ['DPA_PROVIN', 'DPA_DESPRO',
            'DPA_CANTON', 'DPA_DESCAN',
            'DPA_PARROQ', 'DPA_DESPAR']
    inec_ref = dataI[cols].copy()
    
    # Normalizar nombres
    inec_ref['prov_norm']   = inec_ref['DPA_DESPRO'].apply(norm_nombre)
    inec_ref['canton_norm'] = inec_ref['DPA_DESCAN'].apply(norm_nombre)
    inec_ref['parr_norm']   = inec_ref['DPA_DESPAR'].apply(norm_nombre)

    return inec_ref


# ============================================================
# 5. MAPEAR PARROQUIAS A CÓDIGO INEC
# ============================================================

def mapear_parroquias_inec(df_emerg: pd.DataFrame, inec_ref: pd.DataFrame) -> pd.DataFrame:
    """
    Mapea el código de parroquia de df_emerg contra el catálogo INEC.
    
    Si Cod_Parroquia coincide con DPA_PARROQ -> asigna el código.
    Si NO coincide -> coloca NaN.
    """
    # Nos aseguramos que los códigos estén limpios y comparables
    df = df_emerg.copy()
    df['Cod_Parroquia'] = (
        df['Cod_Parroquia']
        .astype(str)
        .str.replace(r'\.0$', '', regex=True)
        .str.strip()
        .str.zfill(6)
    )
    
    inec = inec_ref.copy()
    inec['DPA_PARROQ'] = (
        inec['DPA_PARROQ']
        .astype(str)
        .str.replace(r'\.0$', '', regex=True)
        .str.strip()
        .str.zfill(6)
    )

    # Merge SOLO por código parroquial
    df_geo = df.merge(
        inec[['DPA_PARROQ']],   # solo necesitamos el código oficial
        left_on='Cod_Parroquia',
        right_on='DPA_PARROQ',
        how='left'
    )

    return df_geo



# ============================================================
# 6. REPORTE DE GEOREFERENCIACIÓN
# ============================================================

def reporte_geocodificacion(df_geo: pd.DataFrame) -> None:
    """
    Imprime un resumen de qué tan bien se mapeó a INEC:
    - % de filas con código parroquial asignado
    - ejemplos de parroquias sin match
    """
    total = len(df_geo)
    con_codigo = df_geo['DPA_PARROQ'].notna().sum()
    sin_codigo = total - con_codigo

    print("=== REPORTE GEOREFERENCIACIÓN ===")
    print(f"Filas totales: {total}")
    print(f"Con código INEC (DPA_PARROQ): {con_codigo} ({con_codigo/total*100:.2f}%)")
    print(f"Sin código INEC: {sin_codigo} ({sin_codigo/total*100:.2f}%)")

    if sin_codigo > 0:
        print("\nEjemplos de parroquias SIN match (provincia / cantón / parroquia):")
        ejemplos = (
            df_geo[df_geo['DPA_PARROQ'].isna()]
            [['provincia', 'Canton', 'Parroquia']]
            .drop_duplicates()
            .head(300)
        )
        print(ejemplos)


# ============================================================
# 7. PIPELINE COMPLETO
# ============================================================

def pipeline_georreferenciacion(path_emerg: str, dataI: pd.DataFrame) -> pd.DataFrame:
    """
    Ejecuta todo el flujo:
    1) Carga emergencias
    2) Limpia texto, provincias y Cod_Parroquia
    3) Carga catálogo INEC
    4) Mapea parroquias y agrega DPA_PARROQ
    5) Imprime reporte
    Devuelve df_geo (listo para unir con shapefile).
    """
    print(f"\n{'='*60}")
    print(f"Procesando: {os.path.basename(path_emerg)}")
    print(f"{'='*60}")
    
    print("1) Cargando emergencias...")
    df = load_emergencias(path_emerg)

    print("2) Limpiando emergencias...")
    df_clean = clean_emergencias(df)

    print("3) Cargando codificación INEC...")
    inec_ref = load_inec_codificacion(dataI)

    print("4) Mapeando parroquias a INEC...")
    df_geo = mapear_parroquias_inec(df_clean, inec_ref)

    print("5) Reporte de georreferenciación:")
    reporte_geocodificacion(df_geo)

    return df_geo


# ============================================================
# 8. PROCESAR TODOS LOS ARCHIVOS
# ============================================================

def procesar_todos_emergencias():
    """
    Encuentra todos los archivos emergencias_*.csv y los procesa.
    Guarda cada uno con el nombre: emergencias_X_georreferenciado.csv
    """
    # Cargar el archivo de codificación INEC
    print("Cargando CODIFICACIÓN_2021.xlsx...")
    dataI = pd.read_excel("CODIFICACIÓN_2021.xlsx", header=0, skiprows=1)
    dataI = dataI.dropna(axis=1, how="any")
    print(f"Codificación INEC cargada: {dataI.shape[0]} parroquias")
    
    # Encontrar todos los archivos emergencias_*.csv
    archivos_emergencias = glob.glob("emergencias_*.csv")
    
    if not archivos_emergencias:
        print("\n¡No se encontraron archivos emergencias_*.csv!")
        return
    
    print(f"\n[OK] Se encontraron {len(archivos_emergencias)} archivos para procesar")
    print("Archivos:")
    for archivo in archivos_emergencias:
        print(f"  - {archivo}")
    
    # Procesar cada archivo
    for path_emerg in archivos_emergencias:
        try:
            # Ejecutar el pipeline
            df_geo = pipeline_georreferenciacion(path_emerg, dataI)
            
            # Crear nombre del archivo de salida
            nombre_base = os.path.splitext(os.path.basename(path_emerg))[0]
            output_path = f"{nombre_base}_georreferenciado.csv"
            
            # Guardar resultado
            df_geo.to_csv(output_path, index=False, encoding='utf-8')
            print(f"[OK] Archivo guardado: {output_path}")
            
        except Exception as e:
            print(f"[ERROR] al procesar {path_emerg}: {str(e)}")
            continue
    
    print(f"\n{'='*60}")
    print("[OK] PROCESAMIENTO COMPLETADO")
    print(f"{'='*60}")


# ============================================================
# 9. EJECUCIÓN
# ============================================================

if __name__ == "__main__":
    procesar_todos_emergencias()


Cargando CODIFICACIÓN_2021.xlsx...
Codificación INEC cargada: 1042 parroquias

[OK] Se encontraron 6 archivos para procesar
Archivos:
  - emergencias_agosto_2021.csv
  - emergencias_diciembre_2021.csv
  - emergencias_julio_2021.csv
  - emergencias_noviembre_2021.csv
  - emergencias_octubre_2021.csv
  - emergencias_septiembre_2021.csv

Procesando: emergencias_agosto_2021.csv
1) Cargando emergencias...
2) Limpiando emergencias...
[clean_emergencias] Filas eliminadas por provincia None: 93
3) Cargando codificación INEC...
4) Mapeando parroquias a INEC...
5) Reporte de georreferenciación:
=== REPORTE GEOREFERENCIACIÓN ===
Filas totales: 319896
Con código INEC (DPA_PARROQ): 319883 (100.00%)
Sin código INEC: 13 (0.00%)

Ejemplos de parroquias SIN match (provincia / cantón / parroquia):
       provincia                 Canton            Parroquia
85324      azuay  camilo ponce enriquez  el carmen de pijili
119225  orellana               aguarico             tiputini
[OK] Archivo guardado: eme

C:\Users\JPbau\AppData\Local\Temp\ipykernel_11788\3374631713.py:45: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_csv, sep=";", encoding="utf-8")


2) Limpiando emergencias...
[clean_emergencias] Filas eliminadas por provincia None: 91
3) Cargando codificación INEC...
4) Mapeando parroquias a INEC...
5) Reporte de georreferenciación:
=== REPORTE GEOREFERENCIACIÓN ===
Filas totales: 327729
Con código INEC (DPA_PARROQ): 327722 (100.00%)
Sin código INEC: 7 (0.00%)

Ejemplos de parroquias SIN match (provincia / cantón / parroquia):
      provincia                 Canton            Parroquia
5390      azuay  camilo ponce enriquez  el carmen de pijili
84151  orellana               aguarico             tiputini
[OK] Archivo guardado: emergencias_octubre_2021_georreferenciado.csv

Procesando: emergencias_septiembre_2021.csv
1) Cargando emergencias...
2) Limpiando emergencias...
[clean_emergencias] Filas eliminadas por provincia None: 68
3) Cargando codificación INEC...
4) Mapeando parroquias a INEC...
5) Reporte de georreferenciación:
=== REPORTE GEOREFERENCIACIÓN ===
Filas totales: 307597
Con código INEC (DPA_PARROQ): 307589 (100.00%)
Si

Unir provincias

In [ ]:
import pandas as pd
import glob
import os

def unir_archivos_georreferenciados():
    """
    Une todos los archivos *_georreferenciado.csv en un solo archivo.
    """
    print("="*60)
    print("UNIENDO ARCHIVOS GEORREFERENCIADOS")
    print("="*60)
    
    # Buscar todos los archivos georreferenciados
    archivos = glob.glob("emergencias_*_georreferenciado.csv")
    
    if not archivos:
        print("\n❌ No se encontraron archivos *_georreferenciado.csv")
        return
    
    print(f"\n[OK] Se encontraron {len(archivos)} archivos:")
    for archivo in sorted(archivos):
        tamano_mb = os.path.getsize(archivo) / (1024 * 1024)
        print(f"  - {archivo} ({tamano_mb:.2f} MB)")
    
    # Leer y concatenar todos los archivos
    print(f"\n[PROCESO] Leyendo archivos...")
    dataframes = []
    
    for i, archivo in enumerate(sorted(archivos), 1):
        print(f"  [{i}/{len(archivos)}] Cargando {archivo}...")
        df = pd.read_csv(archivo, encoding='utf-8')
        
        # Agregar columna con el nombre del archivo de origen (opcional)
        df['archivo_origen'] = os.path.basename(archivo)
        
        dataframes.append(df)
        print(f"      [OK] {len(df):,} filas cargadas")
    
    # Concatenar todos los dataframes
    print(f"\n[PROCESO] Concatenando {len(dataframes)} archivos...")
    df_completo = pd.concat(dataframes, ignore_index=True)
    
    print(f"  [OK] Total de filas: {len(df_completo):,}")
    print(f"  [OK] Total de columnas: {len(df_completo.columns)}")
    
    # Mostrar estadísticas
    print(f"\n[ESTADISTICAS]")
    print(f"  - Con codigo INEC: {df_completo['DPA_PARROQ'].notna().sum():,} ({df_completo['DPA_PARROQ'].notna().sum()/len(df_completo)*100:.2f}%)")
    print(f"  - Sin codigo INEC: {df_completo['DPA_PARROQ'].isna().sum():,} ({df_completo['DPA_PARROQ'].isna().sum()/len(df_completo)*100:.2f}%)")
    
    if 'Fecha' in df_completo.columns:
        df_completo['Fecha'] = pd.to_datetime(df_completo['Fecha'], errors='coerce')
        print(f"  - Rango de fechas: {df_completo['Fecha'].min()} a {df_completo['Fecha'].max()}")
    
    # Guardar archivo unificado
    output_file = "emergencias_2021_completo_georreferenciado.csv"
    print(f"\n[GUARDANDO] Archivo unificado: {output_file}")
    df_completo.to_csv(output_file, index=False, encoding='utf-8')
    
    tamano_final_mb = os.path.getsize(output_file) / (1024 * 1024)
    print(f"  [OK] Archivo guardado exitosamente ({tamano_final_mb:.2f} MB)")
    
    # Reporte por archivo de origen
    print(f"\n[DISTRIBUCION] Por archivo:")
    conteo_origen = df_completo['archivo_origen'].value_counts().sort_index()
    for archivo, cantidad in conteo_origen.items():
        print(f"  - {archivo}: {cantidad:,} filas")
    
    print(f"\n{'='*60}")
    print("[OK] PROCESO COMPLETADO")
    print(f"{'='*60}")
    print(f"\nArchivo final: {output_file}")
    print(f"Total de registros: {len(df_completo):,}")
    
    return df_completo

if __name__ == "__main__":
    df_unificado = unir_archivos_georreferenciados()

In [33]:
df_emergencias_total = pd.read_csv("emergencias_2021_completo_georreferenciado.csv")

print(df_emergencias_total.head(10))

# para verlo en forma de tabla 
df_emergencias_total.style.set_properties(**{'text-align': 'left'})



C:\Users\JPbau\AppData\Local\Temp\ipykernel_11788\2669691389.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_emergencias_total = pd.read_csv("emergencias_2021_completo_georreferenciado.csv")


        Fecha provincia     Canton  Cod_Parroquia  \
0  2021-08-07    guayas  guayaquil          90150   
1  2021-08-07    guayas  guayaquil          90150   
2  2021-08-07    guayas  guayaquil          90150   
3  2021-08-07    guayas  guayaquil          90150   
4  2021-08-07    guayas  guayaquil          90150   
5  2021-08-07    guayas  guayaquil          90150   
6  2021-08-07    guayas  guayaquil          90150   
7  2021-08-07    guayas  guayaquil          90150   
8  2021-08-07    guayas  guayaquil          90150   
9  2021-08-07    guayas  guayaquil          90150   

                                           Parroquia               Servicio  \
0  guayaquil, cabecera cantonal y capital provincial    seguridad ciudadana   
1  guayaquil, cabecera cantonal y capital provincial    seguridad ciudadana   
2  guayaquil, cabecera cantonal y capital provincial    seguridad ciudadana   
3  guayaquil, cabecera cantonal y capital provincial    seguridad ciudadana   
4  guayaquil, cabecer

KeyboardInterrupt: 

In [1]:
import pandas as pd

In [5]:
df_agosto = pd.read_csv("emergencias_agosto_2021_georreferenciado.csv")
df_agosto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319896 entries, 0 to 319895
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Fecha          319896 non-null  object 
 1   provincia      319896 non-null  object 
 2   Canton         319896 non-null  object 
 3   Cod_Parroquia  319896 non-null  int64  
 4   Parroquia      319896 non-null  object 
 5   Servicio       319896 non-null  object 
 6   Subtipo        319896 non-null  object 
 7   prov_norm      319896 non-null  object 
 8   canton_norm    319896 non-null  object 
 9   parr_norm      319896 non-null  object 
 10  DPA_PARROQ     319883 non-null  float64
dtypes: float64(1), int64(1), object(9)
memory usage: 26.8+ MB


In [8]:


# Ver las filas donde DPA_PARROQ es NaN (vacío)
filas_sin_codigo = df_agosto[df_agosto['DPA_PARROQ'].isna()]
print(filas_sin_codigo)

# O si quieres verlas una por una
for index, row in filas_sin_codigo.iterrows():
    print(row)

             Fecha provincia                 Canton  Cod_Parroquia  \
85324   2021-08-05     azuay  camilo ponce enriquez          11551   
87044   2021-08-04     azuay  camilo ponce enriquez          11551   
100177  2021-08-11     azuay  camilo ponce enriquez          11551   
119225  2021-08-13  orellana               aguarico         220254   
125696  2021-08-13     azuay  camilo ponce enriquez          11551   
144637  2021-08-06     azuay  camilo ponce enriquez          11551   
175709  2021-08-20     azuay  camilo ponce enriquez          11551   
222088  2021-08-17     azuay  camilo ponce enriquez          11551   
241580  2021-08-23     azuay  camilo ponce enriquez          11551   
247629  2021-08-23     azuay  camilo ponce enriquez          11551   
258977  2021-08-26  orellana               aguarico         220254   
283710  2021-08-29     azuay  camilo ponce enriquez          11551   
284223  2021-08-29     azuay  camilo ponce enriquez          11551   

                  P

# Procesamiento de todos los años 

In [17]:
# ============================================================
# PROCESAMIENTO MULTI-AÑO DE DATOS DE EMERGENCIAS (2022-2025)
# NAVEGACIÓN CON RUTAS RELATIVAS DESDE DataHUB
# ============================================================

import pandas as pd
import sys
import os
import glob
from pathlib import Path

# ============================================================
# DETECTAR Y CONFIGURAR DIRECTORIOS
# ============================================================

# Obtener directorio actual (debería ser DataHUB/2021 donde está el notebook)
DIR_ACTUAL = Path.cwd()

# Navegar al directorio padre (DataHUB)
DIR_DATAHUB = DIR_ACTUAL.parent

print("=" * 80)
print("CONFIGURACIÓN DE DIRECTORIOS")
print("=" * 80)
print(f"📁 Directorio actual (notebook): {DIR_ACTUAL}")
print(f"📁 Directorio DataHUB (padre): {DIR_DATAHUB}")

# Cambiar al directorio DataHUB como base
os.chdir(str(DIR_DATAHUB))
print(f"✅ Directorio de trabajo cambiado a: {Path.cwd()}")
print("=" * 80 + "\n")

# Agregar directorio 2021 al path para importar funciones
sys.path.insert(0, str(DIR_DATAHUB / '2021'))

# Importar funciones de procesamiento
from procesar_todos_emergencias import (
    norm_nombre,
    norm_provincia,
    load_emergencias,
    clean_emergencias,
    load_inec_codificacion,
    mapear_parroquias_inec,
    reporte_geocodificacion,
    pipeline_georreferenciacion
)

# ============================================================
# CONFIGURACIÓN
# ============================================================

# Años a procesar (2021 ya está procesado)
ANIOS = ['2022', '2023', '2024', '2025']

# ============================================================
# FUNCIÓN PRINCIPAL DE PROCESAMIENTO MULTI-AÑO
# ============================================================

def procesar_multianio():
    """
    Procesa todos los archivos de emergencias en los directorios 2022-2025.
    Usa rutas relativas desde DataHUB para navegar entre directorios.
    """
    
    # Verificar que estamos en DataHUB
    dir_base = Path.cwd()
    print(f"📍 Directorio base: {dir_base.name} ({dir_base})\n")
    
    # Estadísticas consolidadas
    estadisticas_totales = []
    
    # Procesar cada año
    for año in ANIOS:
        print("\n" + "=" * 80)
        print(f"PROCESANDO AÑO: {año}")
        print("=" * 80)
        
        # Navegar al directorio del año usando ruta relativa
        dir_año = Path(año)  # Ruta relativa desde DataHUB
        
        # Verificar que el directorio existe
        if not dir_año.exists():
            print(f"⚠️  Directorio no encontrado: {dir_año}")
            print(f"   Ruta buscada: {dir_año.absolute()}")
            print(f"   Saltando año {año}...\n")
            continue
        
        # Construir ruta al archivo de codificación INEC usando ruta relativa
        inec_path = dir_año / f'CODIFICACIÓN_{año}.xlsx'
        
        # Cargar codificación INEC específica del año
        print(f"📂 Cargando codificación INEC desde: {inec_path}")
        
        try:
            dataI = pd.read_excel(str(inec_path), header=0, skiprows=1)
            dataI = dataI.dropna(axis=1, how="any")
            print(f"✅ Codificación INEC {año} cargada: {dataI.shape[0]} parroquias\n")
        except FileNotFoundError:
            print(f"❌ ERROR: No se encontró {inec_path}")
            print(f"   Ruta completa: {inec_path.absolute()}")
            print(f"   Saltando año {año}...\n")
            continue
        except Exception as e:
            print(f"❌ ERROR al cargar INEC {año}: {str(e)}")
            print(f"   Saltando año {año}...\n")
            continue
        
        # Cambiar al directorio del año
        os.chdir(str(dir_año))
        print(f"📁 Navegado a: {año}/ (ruta completa: {Path.cwd()})")
        
        # Buscar archivos de emergencias usando ruta relativa
        archivos_emergencias = glob.glob("emergencias_*.csv")
        
        # Filtrar archivos ya georreferenciados
        archivos_sin_procesar = [
            f for f in archivos_emergencias 
            if not f.endswith('_georreferenciado.csv')
        ]
        
        if not archivos_sin_procesar:
            print(f"ℹ️  No se encontraron archivos sin procesar en {año}/")
            # Volver a DataHUB
            os.chdir(str(dir_base))
            continue
        
        print(f"📊 Archivos encontrados en {año}/: {len(archivos_sin_procesar)}")
        for archivo in archivos_sin_procesar:
            print(f"   - {archivo}")
        print()
        
        # Procesar cada archivo
        archivos_exitosos = 0
        archivos_fallidos = 0
        
        for path_emerg in archivos_sin_procesar:
            try:
                # Ejecutar pipeline de georreferenciación
                df_geo = pipeline_georreferenciacion(path_emerg, dataI)
                
                # Crear nombre del archivo de salida
                nombre_base = os.path.splitext(path_emerg)[0]
                output_path = f"{nombre_base}_georreferenciado.csv"
                
                # Guardar resultado en el mismo directorio (ruta relativa)
                df_geo.to_csv(output_path, index=False, encoding='utf-8')
                print(f"✅ Guardado: {año}/{output_path}\n")
                
                # Guardar estadísticas
                total = len(df_geo)
                con_codigo = df_geo['DPA_PARROQ'].notna().sum()
                estadisticas_totales.append({
                    'año': año,
                    'archivo': path_emerg,
                    'total_registros': total,
                    'georreferenciados': con_codigo,
                    'sin_georreferenciar': total - con_codigo,
                    'porcentaje_exito': (con_codigo / total * 100) if total > 0 else 0
                })
                
                archivos_exitosos += 1
                
            except Exception as e:
                print(f"❌ ERROR al procesar {path_emerg}: {str(e)}\n")
                archivos_fallidos += 1
                continue
        
        print(f"{'=' * 80}")
        print(f"RESUMEN AÑO {año}:")
        print(f"  ✅ Archivos procesados exitosamente: {archivos_exitosos}")
        print(f"  ❌ Archivos con errores: {archivos_fallidos}")
        print(f"{'=' * 80}")
        
        # Volver al directorio DataHUB (ruta relativa)
        os.chdir(str(dir_base))
        print(f"🔙 Volviendo a: DataHUB/ ({Path.cwd()})\n")
    
    # Mostrar estadísticas consolidadas
    if estadisticas_totales:
        print("\n" + "=" * 80)
        print("ESTADÍSTICAS CONSOLIDADAS - TODOS LOS AÑOS")
        print("=" * 80)
        
        df_stats = pd.DataFrame(estadisticas_totales)
        
        # Resumen por año
        print("\n📊 RESUMEN POR AÑO:")
        resumen_año = df_stats.groupby('año').agg({
            'total_registros': 'sum',
            'georreferenciados': 'sum',
            'sin_georreferenciar': 'sum'
        }).reset_index()
        
        resumen_año['% éxito'] = (
            resumen_año['georreferenciados'] / resumen_año['total_registros'] * 100
        ).round(2)
        
        print(resumen_año.to_string(index=False))
        
        # Resumen total
        print("\n📊 RESUMEN TOTAL:")
        total_general = df_stats['total_registros'].sum()
        geo_general = df_stats['georreferenciados'].sum()
        sin_geo_general = df_stats['sin_georreferenciar'].sum()
        
        print(f"  Total de registros procesados: {total_general:,}")
        print(f"  Registros georreferenciados: {geo_general:,} ({geo_general/total_general*100:.2f}%)")
        print(f"  Registros sin georreferenciar: {sin_geo_general:,} ({sin_geo_general/total_general*100:.2f}%)")
        
        # Tabla detallada por archivo
        print("\n📋 DETALLE POR ARCHIVO:")
        df_stats_display = df_stats.copy()
        df_stats_display['porcentaje_exito'] = df_stats_display['porcentaje_exito'].round(2)
        print(df_stats_display.to_string(index=False))
        
    print("\n" + "=" * 80)
    print("✅ PROCESAMIENTO MULTI-AÑO COMPLETADO")
    print("=" * 80)


# ============================================================
# EJECUTAR PROCESAMIENTO
# ============================================================

# Ejecutar la función principal
procesar_multianio()

CONFIGURACIÓN DE DIRECTORIOS
📁 Directorio actual (notebook): c:\
📁 Directorio DataHUB (padre): c:\
✅ Directorio de trabajo cambiado a: c:\

📍 Directorio base:  (c:\)


PROCESANDO AÑO: 2022
⚠️  Directorio no encontrado: 2022
   Ruta buscada: c:\2022
   Saltando año 2022...


PROCESANDO AÑO: 2023
⚠️  Directorio no encontrado: 2023
   Ruta buscada: c:\2023
   Saltando año 2023...


PROCESANDO AÑO: 2024
⚠️  Directorio no encontrado: 2024
   Ruta buscada: c:\2024
   Saltando año 2024...


PROCESANDO AÑO: 2025
⚠️  Directorio no encontrado: 2025
   Ruta buscada: c:\2025
   Saltando año 2025...


✅ PROCESAMIENTO MULTI-AÑO COMPLETADO


#  Todos los tipos de patrones

In [24]:
# ============================================================
# PROCESAMIENTO MULTI-AÑO DE DATOS DE EMERGENCIAS (2022-2025)
# NAVEGACIÓN CON RUTAS RELATIVAS DESDE DataHUB
# ============================================================

import pandas as pd
import sys
import os
import glob
from pathlib import Path

# ============================================================
# DETECTAR Y CONFIGURAR DIRECTORIOS
# ============================================================

# Obtener directorio actual (debería ser DataHUB/2021 donde está el notebook)
# ============================================================
# DETECTAR Y CONFIGURAR DIRECTORIOS
# ============================================================

# Usar ruta fija al directorio DataHUB
DIR_DATAHUB = Path(r'C:\Users\JPbau\OneDrive\Documents\8vo\DataHUB')

# Obtener directorio actual para referencia
DIR_ACTUAL = Path.cwd()

print("=" * 80)
print("CONFIGURACIÓN DE DIRECTORIOS")
print("=" * 80)
print(f"📁 Directorio actual (notebook): {DIR_ACTUAL}")
print(f"📁 Directorio DataHUB (fijo): {DIR_DATAHUB}")

# Verificar que el directorio DataHUB existe
if not DIR_DATAHUB.exists():
    print(f"❌ ERROR: El directorio DataHUB no existe: {DIR_DATAHUB}")
    raise FileNotFoundError(f"No se encontró el directorio: {DIR_DATAHUB}")

# Cambiar al directorio DataHUB como base
os.chdir(str(DIR_DATAHUB))
print(f"✅ Directorio de trabajo cambiado a: {Path.cwd()}")
print("=" * 80 + "\n")

# Agregar directorio 2021 al path para importar funciones
sys.path.insert(0, str(DIR_DATAHUB / '2021'))

# Importar funciones de procesamiento
from procesar_todos_emergencias import (
    norm_nombre,
    norm_provincia,
    load_emergencias,
    clean_emergencias,
    load_inec_codificacion,
    mapear_parroquias_inec,
    reporte_geocodificacion,
    pipeline_georreferenciacion
)

# ============================================================
# CONFIGURACIÓN
# ============================================================

# Años a procesar (2021 ya está procesado)
AÑOS = ['2022', '2023', '2024', '2025']

# Patrones de archivos a buscar
PATRONES_ARCHIVOS = ['emergencias_*.csv', 'incidentes_*.csv']

# ============================================================
# FUNCIÓN PRINCIPAL DE PROCESAMIENTO MULTI-AÑO
# ============================================================

def procesar_multianio():
    """
    Procesa todos los archivos de emergencias e incidentes en los directorios 2022-2025.
    Busca archivos con patrones: emergencias_*.csv e incidentes_*.csv
    Usa rutas relativas desde DataHUB para navegar entre directorios.
    """
    
    # Verificar que estamos en DataHUB
    dir_base = Path.cwd()
    print(f"📍 Directorio base: {dir_base.name} ({dir_base})\n")
    
    # Estadísticas consolidadas
    estadisticas_totales = []
    
    # Procesar cada año
    for año in AÑOS:
        print("\n" + "=" * 80)
        print(f"PROCESANDO AÑO: {año}")
        print("=" * 80)
        
        # Navegar al directorio del año usando ruta relativa
        dir_año = Path(año)  # Ruta relativa desde DataHUB
        
        # Verificar que el directorio existe
        if not dir_año.exists():
            print(f"⚠️  Directorio no encontrado: {dir_año}")
            print(f"   Ruta buscada: {dir_año.absolute()}")
            print(f"   Saltando año {año}...\n")
            continue
        
        # Construir ruta al archivo de codificación INEC usando ruta relativa
        inec_path = dir_año / f'CODIFICACIÓN_{año}.xlsx'
        
        # Cargar codificación INEC específica del año
        print(f"📂 Cargando codificación INEC desde: {inec_path}")
        
        try:
            dataI = pd.read_excel(str(inec_path), header=0, skiprows=1)
            dataI = dataI.dropna(axis=1, how="any")
            print(f"✅ Codificación INEC {año} cargada: {dataI.shape[0]} parroquias\n")
        except FileNotFoundError:
            print(f"❌ ERROR: No se encontró {inec_path}")
            print(f"   Ruta completa: {inec_path.absolute()}")
            print(f"   Saltando año {año}...\n")
            continue
        except Exception as e:
            print(f"❌ ERROR al cargar INEC {año}: {str(e)}")
            print(f"   Saltando año {año}...\n")
            continue
        
        # Cambiar al directorio del año
        os.chdir(str(dir_año))
        print(f"📁 Navegado a: {año}/ (ruta completa: {Path.cwd()})")
        
        # Buscar archivos con múltiples patrones
        print(f"\n🔍 Buscando archivos con patrones: {', '.join(PATRONES_ARCHIVOS)}")
        archivos_encontrados = []
        
        for patron in PATRONES_ARCHIVOS:
            archivos = glob.glob(patron)
            if archivos:
                print(f"   - Patrón '{patron}': {len(archivos)} archivo(s)")
                archivos_encontrados.extend(archivos)
        
        # Filtrar archivos ya georreferenciados
        archivos_sin_procesar = [
            f for f in archivos_encontrados 
            if not f.endswith('_georreferenciado.csv')
        ]
        
        # Eliminar duplicados (por si acaso)
        archivos_sin_procesar = list(set(archivos_sin_procesar))
        
        if not archivos_sin_procesar:
            print(f"ℹ️  No se encontraron archivos sin procesar en {año}/")
            # Volver a DataHUB
            os.chdir(str(dir_base))
            continue
        
        print(f"\n📊 Total de archivos a procesar en {año}/: {len(archivos_sin_procesar)}")
        for archivo in sorted(archivos_sin_procesar):
            print(f"   - {archivo}")
        print()
        
        # Procesar cada archivo
        archivos_exitosos = 0
        archivos_fallidos = 0
        
        for path_emerg in archivos_sin_procesar:
            try:
                # Ejecutar pipeline de georreferenciación
                df_geo = pipeline_georreferenciacion(path_emerg, dataI)
                
                # Crear nombre del archivo de salida
                nombre_base = os.path.splitext(path_emerg)[0]
                output_path = f"{nombre_base}_georreferenciado.csv"
                
                # Guardar resultado en el mismo directorio (ruta relativa)
                df_geo.to_csv(output_path, index=False, encoding='utf-8')
                print(f"✅ Guardado: {año}/{output_path}\n")
                
                # Guardar estadísticas
                total = len(df_geo)
                con_codigo = df_geo['DPA_PARROQ'].notna().sum()
                estadisticas_totales.append({
                    'año': año,
                    'archivo': path_emerg,
                    'total_registros': total,
                    'georreferenciados': con_codigo,
                    'sin_georreferenciar': total - con_codigo,
                    'porcentaje_exito': (con_codigo / total * 100) if total > 0 else 0
                })
                
                archivos_exitosos += 1
                
            except Exception as e:
                print(f"❌ ERROR al procesar {path_emerg}: {str(e)}\n")
                archivos_fallidos += 1
                continue
        
        print(f"{'=' * 80}")
        print(f"RESUMEN AÑO {año}:")
        print(f"  ✅ Archivos procesados exitosamente: {archivos_exitosos}")
        print(f"  ❌ Archivos con errores: {archivos_fallidos}")
        print(f"{'=' * 80}")
        
        # Volver al directorio DataHUB (ruta relativa)
        os.chdir(str(dir_base))
        print(f"🔙 Volviendo a: DataHUB/ ({Path.cwd()})\n")
    
    # Mostrar estadísticas consolidadas
    if estadisticas_totales:
        print("\n" + "=" * 80)
        print("ESTADÍSTICAS CONSOLIDADAS - TODOS LOS AÑOS")
        print("=" * 80)
        
        df_stats = pd.DataFrame(estadisticas_totales)
        
        # Resumen por año
        print("\n📊 RESUMEN POR AÑO:")
        resumen_año = df_stats.groupby('año').agg({
            'total_registros': 'sum',
            'georreferenciados': 'sum',
            'sin_georreferenciar': 'sum'
        }).reset_index()
        
        resumen_año['% éxito'] = (
            resumen_año['georreferenciados'] / resumen_año['total_registros'] * 100
        ).round(2)
        
        print(resumen_año.to_string(index=False))
        
        # Resumen total
        print("\n📊 RESUMEN TOTAL:")
        total_general = df_stats['total_registros'].sum()
        geo_general = df_stats['georreferenciados'].sum()
        sin_geo_general = df_stats['sin_georreferenciar'].sum()
        
        print(f"  Total de registros procesados: {total_general:,}")
        print(f"  Registros georreferenciados: {geo_general:,} ({geo_general/total_general*100:.2f}%)")
        print(f"  Registros sin georreferenciar: {sin_geo_general:,} ({sin_geo_general/total_general*100:.2f}%)")
        
        # Tabla detallada por archivo
        print("\n📋 DETALLE POR ARCHIVO:")
        df_stats_display = df_stats.copy()
        df_stats_display['porcentaje_exito'] = df_stats_display['porcentaje_exito'].round(2)
        print(df_stats_display.to_string(index=False))
        
    print("\n" + "=" * 80)
    print("✅ PROCESAMIENTO MULTI-AÑO COMPLETADO")
    print("=" * 80)


# ============================================================
# EJECUTAR PROCESAMIENTO
# ============================================================

# Ejecutar la función principal
procesar_multianio()

CONFIGURACIÓN DE DIRECTORIOS
📁 Directorio actual (notebook): c:\
📁 Directorio DataHUB (fijo): C:\Users\JPbau\OneDrive\Documents\8vo\DataHUB
✅ Directorio de trabajo cambiado a: C:\Users\JPbau\OneDrive\Documents\8vo\DataHUB

📍 Directorio base: DataHUB (C:\Users\JPbau\OneDrive\Documents\8vo\DataHUB)


PROCESANDO AÑO: 2022
📂 Cargando codificación INEC desde: 2022\CODIFICACIÓN_2022.xlsx
✅ Codificación INEC 2022 cargada: 1042 parroquias

📁 Navegado a: 2022/ (ruta completa: C:\Users\JPbau\OneDrive\Documents\8vo\DataHUB\2022)

🔍 Buscando archivos con patrones: emergencias_*.csv, incidentes_*.csv
   - Patrón 'emergencias_*.csv': 12 archivo(s)

📊 Total de archivos a procesar en 2022/: 12
   - emergencias_abril_2022.csv
   - emergencias_agosto_2022.csv
   - emergencias_diciembre_2022.csv
   - emergencias_enero_2022.csv
   - emergencias_febrero_2022.csv
   - emergencias_julio_2022.csv
   - emergencias_junio_2022.csv
   - emergencias_marzo_2022.csv
   - emergencias_mayo_2022.csv
   - emergencias_no

In [8]:
# Ejecuta esto en una celda para ver qué columnas tiene CODIFICACIÓN_2025.xlsx
import pandas as pd
ruta_2025 = r'C:\Users\JPbau\OneDrive\Documents\8vo\DataHUB\2025\CODIFICACIÓN_2025.xlsx'
df_2025 = pd.read_excel(ruta_2025, header=0, skiprows=1)
print("Columnas en CODIFICACIÓN_2025.xlsx:")
print(df_2025.columns.tolist())

Columnas en CODIFICACIÓN_2025.xlsx:
['Unnamed: 0', 'DPA_PROVIN', 'DPA_DESPRO', 'DPA_CANTON', 'DPA_DESCAN', 'DPA_PARROQ', 'DPA_DESPAR']


# Ultimo intento


In [ ]:
# PROCESAMIENTO MULTI-AÑO DE DATOS DE EMERGENCIAS (2022-2025)
# NAVEGACIÓN CON RUTAS RELATIVAS DESDE DataHUB
# ============================================================

import pandas as pd
import sys
import os
import glob
from pathlib import Path

# ============================================================
# DETECTAR Y CONFIGURAR DIRECTORIOS
# ============================================================

# Usar ruta fija al directorio DataHUB
DIR_DATAHUB = Path(r'C:\Users\JPbau\OneDrive\Documents\8vo\DataHUB')

# Obtener directorio actual para referencia
DIR_ACTUAL = Path.cwd()

print("=" * 80)
print("CONFIGURACIÓN DE DIRECTORIOS")
print("=" * 80)
print(f"📁 Directorio actual (notebook): {DIR_ACTUAL}")
print(f"📁 Directorio DataHUB (fijo): {DIR_DATAHUB}")

# Verificar que el directorio DataHUB existe
if not DIR_DATAHUB.exists():
    print(f"❌ ERROR: El directorio DataHUB no existe: {DIR_DATAHUB}")
    raise FileNotFoundError(f"No se encontró el directorio: {DIR_DATAHUB}")

# Cambiar al directorio DataHUB como base
os.chdir(str(DIR_DATAHUB))
print(f"✅ Directorio de trabajo cambiado a: {Path.cwd()}")
print("=" * 80 + "\n")

# Agregar directorio 2021 al path para importar funciones
sys.path.insert(0, str(DIR_DATAHUB / '2021'))

from procesar_todos_emergencias import (
    norm_nombre,
    norm_provincia,
    load_emergencias,
    clean_emergencias,
    mapear_parroquias_inec,
    reporte_geocodificacion,
    pipeline_georreferenciacion
)

# ============================================================
# CONFIGURACIÓN
# ============================================================

AÑOS = ['2022', '2023', '2024', '2025']
PATRONES_ARCHIVOS = ['emergencias_*.csv', 'incidentes_*.csv']

# ============================================================
# FUNCIÓN PRINCIPAL DE PROCESAMIENTO MULTI-AÑO
# ============================================================

def procesar_multianio():
    """
    Procesa todos los archivos de emergencias e incidentes en los directorios 2022-2025.
    Busca archivos con patrones: emergencias_*.csv e incidentes_*.csv
    Usa rutas relativas desde DataHUB para navegar entre directorios.
    """
    
    dir_base = Path.cwd()
    print(f"📍 Directorio base: {dir_base.name} ({dir_base})\n")
    
    estadisticas_totales = []
    
    for año in AÑOS:
        print("\n" + "=" * 80)
        print(f"PROCESANDO AÑO: {año}")
        print("=" * 80)
        
        dir_año = Path(año)
        
        if not dir_año.exists():
            print(f"⚠️  Directorio no encontrado: {dir_año}")
            print(f"   Ruta buscada: {dir_año.absolute()}")
            print(f"   Saltando año {año}...\n")
            continue
        
        inec_path = dir_año / f'CODIFICACIÓN_{año}.xlsx'
        
        print(f"📂 Cargando codificación INEC desde: {inec_path}")
        
        try:
            # Cargar archivo Excel
            dataI = pd.read_excel(str(inec_path), header=0, skiprows=1)
            
            # Eliminar columna Unnamed: 0 si existe
            if 'Unnamed: 0' in dataI.columns:
                dataI = dataI.drop(columns=['Unnamed: 0'])
            
            # Verificar columnas necesarias
            cols_necesarias = ['DPA_PROVIN', 'DPA_DESPRO', 'DPA_CANTON', 
                             'DPA_DESCAN', 'DPA_PARROQ', 'DPA_DESPAR']
            
            cols_faltantes = [col for col in cols_necesarias if col not in dataI.columns]
            if cols_faltantes:
                print(f"   ❌ Columnas faltantes: {cols_faltantes}")
                print(f"   📋 Columnas disponibles: {dataI.columns.tolist()}")
                raise ValueError(f"Faltan columnas en CODIFICACIÓN_{año}.xlsx")
            
            # Seleccionar solo columnas necesarias
            dataI = dataI[cols_necesarias].copy()
            
            # Normalizar nombres
            dataI['prov_norm']   = dataI['DPA_DESPRO'].apply(norm_nombre)
            dataI['canton_norm'] = dataI['DPA_DESCAN'].apply(norm_nombre)
            dataI['parr_norm']   = dataI['DPA_DESPAR'].apply(norm_nombre)
            
            print(f"✅ Codificación INEC {año} cargada: {dataI.shape[0]} parroquias\n")
            
        except FileNotFoundError:
            print(f"❌ ERROR: No se encontró {inec_path}")
            print(f"   Ruta completa: {inec_path.absolute()}")
            print(f"   Saltando año {año}...\n")
            continue
        except Exception as e:
            print(f"❌ ERROR al cargar INEC {año}: {str(e)}")
            print(f"   Saltando año {año}...\n")
            continue
        
        os.chdir(str(dir_año))
        print(f"📁 Navegado a: {año}/ (ruta completa: {Path.cwd()})")
        
        print(f"\n🔍 Buscando archivos con patrones: {', '.join(PATRONES_ARCHIVOS)}")
        archivos_encontrados = []
        
        for patron in PATRONES_ARCHIVOS:
            archivos = glob.glob(patron)
            if archivos:
                print(f"   - Patrón '{patron}': {len(archivos)} archivo(s)")
                archivos_encontrados.extend(archivos)
        
        archivos_sin_procesar = [
            f for f in archivos_encontrados 
            if not f.endswith('_georreferenciado.csv')
        ]
        archivos_sin_procesar = list(set(archivos_sin_procesar))
        
        if not archivos_sin_procesar:
            print(f"ℹ️  No se encontraron archivos sin procesar en {año}/")
            os.chdir(str(dir_base))
            continue
        
        print(f"\n📊 Total de archivos a procesar en {año}/: {len(archivos_sin_procesar)}")
        for archivo in sorted(archivos_sin_procesar):
            print(f"   - {archivo}")
        print()
        
        archivos_exitosos = 0
        archivos_fallidos = 0
        
        for path_emerg in archivos_sin_procesar:
            try:
                df_geo = pipeline_georreferenciacion(path_emerg, dataI)
                
                nombre_base = os.path.splitext(path_emerg)[0]
                output_path = f"{nombre_base}_georreferenciado.csv"
                
                df_geo.to_csv(output_path, index=False, encoding='utf-8')
                print(f"✅ Guardado: {año}/{output_path}\n")
                
                total = len(df_geo)
                con_codigo = df_geo['DPA_PARROQ'].notna().sum()
                estadisticas_totales.append({
                    'año': año,
                    'archivo': path_emerg,
                    'total_registros': total,
                    'georreferenciados': con_codigo,
                    'sin_georreferenciar': total - con_codigo,
                    'porcentaje_exito': (con_codigo / total * 100) if total > 0 else 0
                })
                
                archivos_exitosos += 1
                
            except Exception as e:
                print(f"❌ ERROR al procesar {path_emerg}: {str(e)}\n")
                archivos_fallidos += 1
                continue
        
        print(f"{'=' * 80}")
        print(f"RESUMEN AÑO {año}:")
        print(f"  ✅ Archivos procesados exitosamente: {archivos_exitosos}")
        print(f"  ❌ Archivos con errores: {archivos_fallidos}")
        print(f"{'=' * 80}")
        
        os.chdir(str(dir_base))
        print(f"🔙 Volviendo a: DataHUB/ ({Path.cwd()})\n")
    
    if estadisticas_totales:
        print("\n" + "=" * 80)
        print("ESTADÍSTICAS CONSOLIDADAS - TODOS LOS AÑOS")
        print("=" * 80)
        
        df_stats = pd.DataFrame(estadisticas_totales)
        
        print("\n📊 RESUMEN POR AÑO:")
        resumen_año = df_stats.groupby('año').agg({
            'total_registros': 'sum',
            'georreferenciados': 'sum',
            'sin_georreferenciar': 'sum'
        }).reset_index()
        
        resumen_año['% éxito'] = (
            resumen_año['georreferenciados'] / resumen_año['total_registros'] * 100
        ).round(2)
        
        print(resumen_año.to_string(index=False))
        
        print("\n📊 RESUMEN TOTAL:")
        total_general = df_stats['total_registros'].sum()
        geo_general = df_stats['georreferenciados'].sum()
        sin_geo_general = df_stats['sin_georreferenciar'].sum()
        
        print(f"  Total de registros procesados: {total_general:,}")
        print(f"  Registros georreferenciados: {geo_general:,} ({geo_general/total_general*100:.2f}%)")
        print(f"  Registros sin georreferenciar: {sin_geo_general:,} ({sin_geo_general/total_general*100:.2f}%)")
        
        print("\n📋 DETALLE POR ARCHIVO:")
        df_stats_display = df_stats.copy()
        df_stats_display['porcentaje_exito'] = df_stats_display['porcentaje_exito'].round(2)
        print(df_stats_display.to_string(index=False))
        
    print("\n" + "=" * 80)
    print("✅ PROCESAMIENTO MULTI-AÑO COMPLETADO")
    print("=" * 80)


# ============================================================
# EJECUTAR PROCESAMIENTO
# ============================================================

procesar_multianio()

CONFIGURACIÓN DE DIRECTORIOS
📁 Directorio actual (notebook): C:\Users\JPbau\OneDrive\Documents\8vo\DataHUB
📁 Directorio DataHUB (fijo): C:\Users\JPbau\OneDrive\Documents\8vo\DataHUB
✅ Directorio de trabajo cambiado a: C:\Users\JPbau\OneDrive\Documents\8vo\DataHUB

📍 Directorio base: DataHUB (C:\Users\JPbau\OneDrive\Documents\8vo\DataHUB)


PROCESANDO AÑO: 2022
📂 Cargando codificación INEC desde: 2022\CODIFICACIÓN_2022.xlsx
✅ Codificación INEC 2022 cargada: 1042 parroquias

📁 Navegado a: 2022/ (ruta completa: C:\Users\JPbau\OneDrive\Documents\8vo\DataHUB\2022)

🔍 Buscando archivos con patrones: emergencias_*.csv, incidentes_*.csv
   - Patrón 'emergencias_*.csv': 12 archivo(s)

📊 Total de archivos a procesar en 2022/: 12
   - emergencias_abril_2022.csv
   - emergencias_agosto_2022.csv
   - emergencias_diciembre_2022.csv
   - emergencias_enero_2022.csv
   - emergencias_febrero_2022.csv
   - emergencias_julio_2022.csv
   - emergencias_junio_2022.csv
   - emergencias_marzo_2022.csv
   - eme

In [26]:
# Deseo unir todos los archivos emergencias georeferenciadas en un solo archivo de todos los anios
from unir_georreferenciados import unir_archivos_georreferenciados

# ============================================================
# UNIR ARCHIVOS GEORREFERENCIADOS POR AÑO
# ============================================================

import pandas as pd
import glob
import os
from pathlib import Path

# ============================================================
# CONFIGURACIÓN
# ============================================================

# Ruta fija al directorio DataHUB
DIR_DATAHUB = Path(r'C:\Users\JPbau\OneDrive\Documents\8vo\DataHUB')

# Años a procesar
AÑOS = ['2022', '2023', '2024', '2025']

# Patrones de archivos georreferenciados a buscar
PATRONES = ['emergencias_*_georreferenciado.csv', 'incidentes_*_georreferenciado.csv']

# ============================================================
# FUNCIÓN PARA UNIR ARCHIVOS DE UN AÑO
# ============================================================

def unir_archivos_por_año(año: str, dir_año: Path):
    """
    Une todos los archivos georreferenciados de un año específico en un solo archivo.
    
    Args:
        año: Año a procesar (ej: '2022')
        dir_año: Path al directorio del año
    """
    print("\n" + "=" * 80)
    print(f"UNIENDO ARCHIVOS GEORREFERENCIADOS - AÑO {año}")
    print("=" * 80)
    
    # Cambiar al directorio del año
    os.chdir(str(dir_año))
    
    # Buscar archivos georreferenciados con múltiples patrones
    archivos = []
    for patron in PATRONES:
        archivos.extend(glob.glob(patron))
    
    # Eliminar duplicados
    archivos = list(set(archivos))
    
    if not archivos:
        print(f"⚠️  No se encontraron archivos georreferenciados en {año}/")
        return None
    
    print(f"\n✅ Se encontraron {len(archivos)} archivos:")
    for archivo in sorted(archivos):
        tamano_mb = os.path.getsize(archivo) / (1024 * 1024)
        print(f"   - {archivo} ({tamano_mb:.2f} MB)")
    
    # Leer y concatenar todos los archivos
    print(f"\n📥 Cargando archivos...")
    dataframes = []
    
    for i, archivo in enumerate(sorted(archivos), 1):
        print(f"   [{i}/{len(archivos)}] {archivo}...", end=" ")
        try:
            df = pd.read_csv(archivo, encoding='utf-8')
            
            # Agregar columna con el nombre del archivo de origen
            df['archivo_origen'] = os.path.basename(archivo)
            
            dataframes.append(df)
            print(f"✅ {len(df):,} filas")
        except Exception as e:
            print(f"❌ Error: {str(e)}")
            continue
    
    if not dataframes:
        print(f"❌ No se pudo cargar ningún archivo")
        return None
    
    # Concatenar todos los dataframes
    print(f"\n🔗 Concatenando {len(dataframes)} archivos...")
    df_completo = pd.concat(dataframes, ignore_index=True)
    
    print(f"   ✅ Total de filas: {len(df_completo):,}")
    print(f"   ✅ Total de columnas: {len(df_completo.columns)}")
    
    # Mostrar estadísticas
    print(f"\n📊 ESTADÍSTICAS:")
    con_codigo = df_completo['DPA_PARROQ'].notna().sum()
    sin_codigo = df_completo['DPA_PARROQ'].isna().sum()
    print(f"   - Con código INEC: {con_codigo:,} ({con_codigo/len(df_completo)*100:.2f}%)")
    print(f"   - Sin código INEC: {sin_codigo:,} ({sin_codigo/len(df_completo)*100:.2f}%)")
    
    if 'Fecha' in df_completo.columns:
        df_completo['Fecha'] = pd.to_datetime(df_completo['Fecha'], errors='coerce')
        fecha_min = df_completo['Fecha'].min()
        fecha_max = df_completo['Fecha'].max()
        if pd.notna(fecha_min) and pd.notna(fecha_max):
            print(f"   - Rango de fechas: {fecha_min.date()} a {fecha_max.date()}")
    
    # Guardar archivo unificado
    output_file = f"eventos_{año}_completo_georreferenciado.csv"
    print(f"\n💾 Guardando archivo unificado: {output_file}")
    df_completo.to_csv(output_file, index=False, encoding='utf-8')
    
    tamano_final_mb = os.path.getsize(output_file) / (1024 * 1024)
    print(f"   ✅ Guardado exitosamente ({tamano_final_mb:.2f} MB)")
    
    # Reporte por archivo de origen
    print(f"\n📋 DISTRIBUCIÓN POR ARCHIVO:")
    conteo_origen = df_completo['archivo_origen'].value_counts().sort_index()
    for archivo, cantidad in conteo_origen.items():
        print(f"   - {archivo}: {cantidad:,} filas")
    
    print(f"\n{'=' * 80}")
    print(f"✅ AÑO {año} COMPLETADO")
    print(f"{'=' * 80}")
    
    return df_completo


# ============================================================
# FUNCIÓN PRINCIPAL - UNIR TODOS LOS AÑOS
# ============================================================

def unir_todos_los_años():
    """
    Une archivos georreferenciados para todos los años (2022-2025).
    """
    # Guardar directorio original
    dir_original = Path.cwd()
    
    print("\n" + "=" * 80)
    print("UNIFICACIÓN MULTI-AÑO DE ARCHIVOS GEORREFERENCIADOS")
    print("=" * 80)
    print(f"📁 Directorio DataHUB: {DIR_DATAHUB}\n")
    
    # Verificar que DataHUB existe
    if not DIR_DATAHUB.exists():
        print(f"❌ ERROR: No se encontró el directorio DataHUB: {DIR_DATAHUB}")
        return
    
    # Resumen de resultados
    resumen = []
    
    # Procesar cada año
    for año in AÑOS:
        dir_año = DIR_DATAHUB / año
        
        if not dir_año.exists():
            print(f"\n⚠️  Directorio no encontrado: {año}/")
            continue
        
        try:
            # Unir archivos del año
            df_año = unir_archivos_por_año(año, dir_año)
            
            if df_año is not None:
                resumen.append({
                    'año': año,
                    'total_registros': len(df_año),
                    'georreferenciados': df_año['DPA_PARROQ'].notna().sum(),
                    'sin_georreferenciar': df_año['DPA_PARROQ'].isna().sum(),
                    'archivos_unidos': len(df_año['archivo_origen'].unique()),
                    'archivo_salida': f"eventos_{año}_completo_georreferenciado.csv"
                })
        
        except Exception as e:
            print(f"\n❌ ERROR al procesar año {año}: {str(e)}")
            continue
        
        # Volver al directorio original
        os.chdir(str(dir_original))
    
    # Mostrar resumen final
    if resumen:
        print("\n\n" + "=" * 80)
        print("RESUMEN FINAL - TODOS LOS AÑOS")
        print("=" * 80)
        
        df_resumen = pd.DataFrame(resumen)
        
        # Calcular porcentaje de éxito
        df_resumen['% éxito'] = (
            df_resumen['georreferenciados'] / df_resumen['total_registros'] * 100
        ).round(2)
        
        print("\n📊 ESTADÍSTICAS POR AÑO:")
        print(df_resumen[['año', 'total_registros', 'georreferenciados', 
                          'sin_georreferenciar', '% éxito', 'archivos_unidos']].to_string(index=False))
        
        # Totales generales
        print("\n📊 TOTALES GENERALES:")
        total_registros = df_resumen['total_registros'].sum()
        total_georreferenciados = df_resumen['georreferenciados'].sum()
        total_sin_georreferenciar = df_resumen['sin_georreferenciar'].sum()
        
        print(f"   Total de registros: {total_registros:,}")
        print(f"   Georreferenciados: {total_georreferenciados:,} ({total_georreferenciados/total_registros*100:.2f}%)")
        print(f"   Sin georreferenciar: {total_sin_georreferenciar:,} ({total_sin_georreferenciar/total_registros*100:.2f}%)")
        
        # Archivos generados
        print("\n📁 ARCHIVOS GENERADOS:")
        for idx, row in df_resumen.iterrows():
            print(f"   - {row['año']}/{row['archivo_salida']}")
        
    print("\n" + "=" * 80)
    print("✅ PROCESO COMPLETADO")
    print("=" * 80)


# ============================================================
# EJECUTAR
# ============================================================

unir_todos_los_años()



UNIFICACIÓN MULTI-AÑO DE ARCHIVOS GEORREFERENCIADOS
📁 Directorio DataHUB: C:\Users\JPbau\OneDrive\Documents\8vo\DataHUB


UNIENDO ARCHIVOS GEORREFERENCIADOS - AÑO 2022

✅ Se encontraron 12 archivos:
   - emergencias_abril_2022_georreferenciado.csv (57.54 MB)
   - emergencias_agosto_2022_georreferenciado.csv (54.66 MB)
   - emergencias_diciembre_2022_georreferenciado.csv (55.77 MB)
   - emergencias_enero_2022_georreferenciado.csv (55.94 MB)
   - emergencias_febrero_2022_georreferenciado.csv (54.16 MB)
   - emergencias_julio_2022_georreferenciado.csv (57.07 MB)
   - emergencias_junio_2022_georreferenciado.csv (54.03 MB)
   - emergencias_marzo_2022_georreferenciado.csv (57.74 MB)
   - emergencias_mayo_2022_georreferenciado.csv (59.16 MB)
   - emergencias_noviembre_2022_georreferenciado.csv (53.06 MB)
   - emergencias_octubre_2022_georreferenciado.csv (56.46 MB)
   - emergencias_septiembre_2022_georreferenciado.csv (54.23 MB)

📥 Cargando archivos...
   [1/12] emergencias_abril_2022_georre

C:\Users\JPbau\AppData\Local\Temp\ipykernel_29028\3833260322.py:69: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo, encoding='utf-8')


✅ 269,501 filas
   [3/12] incidentes_diciembre2024_georreferenciado.csv... ✅ 297,251 filas
   [4/12] incidentes_enero_2024_georreferenciado.csv... ✅ 208,143 filas
   [5/12] incidentes_febrero_2024_georreferenciado.csv... ✅ 276,756 filas
   [6/12] incidentes_julio_2024_georreferenciado.csv... 

C:\Users\JPbau\AppData\Local\Temp\ipykernel_29028\3833260322.py:69: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo, encoding='utf-8')


✅ 268,214 filas
   [7/12] incidentes_junio_2024_georreferenciado.csv... ✅ 277,003 filas
   [8/12] incidentes_marzo_2024_georreferenciado.csv... ✅ 291,103 filas
   [9/12] incidentes_mayo_2024_georreferenciado.csv... ✅ 278,041 filas
   [10/12] incidentes_noviembre2024_georreferenciado.csv... ✅ 279,528 filas
   [11/12] incidentes_octubre2024_georreferenciado.csv... ✅ 294,423 filas
   [12/12] incidentes_septiembre2024_georreferenciado.csv... ✅ 279,880 filas

🔗 Concatenando 12 archivos...
   ✅ Total de filas: 3,291,743
   ✅ Total de columnas: 12

📊 ESTADÍSTICAS:
   - Con código INEC: 3,291,622 (100.00%)
   - Sin código INEC: 121 (0.00%)
   - Rango de fechas: 2024-01-01 a 2024-12-31

💾 Guardando archivo unificado: eventos_2024_completo_georreferenciado.csv
   ✅ Guardado exitosamente (748.36 MB)

📋 DISTRIBUCIÓN POR ARCHIVO:
   - incidentes_abril_2024_georreferenciado.csv: 271,900 filas
   - incidentes_agosto_2024_georreferenciado.csv: 269,501 filas
   - incidentes_diciembre2024_georreferencia

----------------------------------------

Limpieza de datos(eventos georeferenciados por anio) usando la librería de polars 

In [3]:
#polars 
%pip install polars


   ---------------------------------------- 0.0/805.6 kB ? eta -:--:--
   ------------- -------------------------- 262.1/805.6 kB ? eta -:--:--
   ---------------------------------------- 805.6/805.6 kB 2.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/45.0 MB ? eta -:--:--
   ---------------------------------------- 0.5/45.0 MB 2.8 MB/s eta 0:00:16
    --------------------------------------- 1.0/45.0 MB 2.5 MB/s eta 0:00:18
   - -------------------------------------- 1.6/45.0 MB 2.7 MB/s eta 0:00:17
   -- ------------------------------------- 2.4/45.0 MB 2.9 MB/s eta 0:00:15
   -- ------------------------------------- 2.9/45.0 MB 2.8 MB/s eta 0:00:15
   --- ------------------------------------ 3.4/45.0 MB 3.0 MB/s eta 0:00:15
   --- ------------------------------------ 4.2/45.0 MB 2.9 MB/s eta 0:00:15
   ---- ----------------------------------- 4.7/45.0 MB 2.9 MB/s eta 0:00:14
   ---- ----------------------------------- 5.2/45.0 MB 2.8 MB/s eta 0:00:15
   ---- -----


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from pathlib import Path  
import pandas as pd 

In [4]:
import polars as pl
import glob
import os

def concatenar_archivos_georreferenciados():
    """
    Concatena todos los archivos georreferenciados de eventos y emergencias
    de los años 2021-2025 usando Polars, manejando errores de parsing y 
    diferencias en columnas.
    """
    print("="*80)
    print("CONCATENANDO ARCHIVOS GEORREFERENCIADOS CON POLARS")
    print("="*80)
    
    # Buscar archivos en todos los años
    patrones = [
        "emergencias_*_completo_georreferenciado.csv",
        "eventos_*_completo_georreferenciado.csv",
        "../2022/emergencias_*_completo_georreferenciado.csv",
        "../2022/eventos_*_completo_georreferenciado.csv",
        "../2023/emergencias_*_completo_georreferenciado.csv",
        "../2023/eventos_*_completo_georreferenciado.csv",
        "../2024/emergencias_*_completo_georreferenciado.csv",
        "../2024/eventos_*_completo_georreferenciado.csv",
        "../2025/emergencias_*_completo_georreferenciado.csv",
        "../2025/eventos_*_completo_georreferenciado.csv",
    ]
    
    archivos = []
    for patron in patrones:
        archivos.extend(glob.glob(patron))
    
    if not archivos:
        print("\n❌ No se encontraron archivos georreferenciados")
        return None
    
    print(f"\n✅ Se encontraron {len(archivos)} archivos:")
    for archivo in sorted(archivos):
        tamano_mb = os.path.getsize(archivo) / (1024 * 1024)
        print(f"  • {archivo} ({tamano_mb:.2f} MB)")
    
    # Leer archivos con Polars
    dataframes = []
    columnas_comunes = None
    
    for i, archivo in enumerate(sorted(archivos), 1):
        print(f"\n[{i}/{len(archivos)}] Procesando {os.path.basename(archivo)}...")
        
        try:
            # Leer con columnas como strings para evitar errores de parsing
            df_temp = pl.read_csv(
                archivo,
                encoding='utf-8',
                infer_schema_length=0,  # Leer todo como string primero
                ignore_errors=True
            )
            
            print(f"  ✓ Leído: {df_temp.shape[0]:,} filas, {df_temp.shape[1]} columnas")
            
            # Identificar columnas comunes
            if columnas_comunes is None:
                columnas_comunes = set(df_temp.columns)
            else:
                columnas_comunes = columnas_comunes.intersection(set(df_temp.columns))
            
            # Convertir columnas numéricas con manejo de errores
            for col in df_temp.columns:
                if col in ['Cod_Provincia', 'Cod_Canton', 'Cod_Parroquia']:
                    # Limpiar valores tipo "000nan" y convertir
                    df_temp = df_temp.with_columns(
                        pl.when(pl.col(col).str.contains("nan"))
                        .then(None)
                        .otherwise(pl.col(col))
                        .cast(pl.Int64, strict=False)
                        .alias(col)
                    )
            
            # Agregar columna de origen
            df_temp = df_temp.with_columns(
                pl.lit(os.path.basename(archivo)).alias("archivo_origen")
            )
            
            dataframes.append(df_temp)
            print(f"  ✓ Procesado correctamente")
            
        except Exception as e:
            print(f"  ❌ Error al leer {archivo}: {e}")
            continue
    
    if not dataframes:
        print("\n❌ No se pudieron leer archivos")
        return None
    
    print(f"\n{'='*80}")
    print(f"CONCATENANDO {len(dataframes)} DATAFRAMES")
    print(f"{'='*80}")
    
    # Asegurar que todos tengan las mismas columnas base
    print(f"\n📋 Columnas comunes encontradas: {len(columnas_comunes)}")
    
    # Seleccionar solo columnas comunes + archivo_origen de cada dataframe
    dataframes_normalizados = []
    for df in dataframes:
        cols_seleccionar = [col for col in df.columns if col in columnas_comunes or col == "archivo_origen"]
        df_normalizado = df.select(cols_seleccionar)
        dataframes_normalizados.append(df_normalizado)
    
    # Concatenar usando vertical_relaxed para mayor flexibilidad
    try:
        df_final = pl.concat(dataframes_normalizados, how='vertical_relaxed')
        print(f"\n✅ DataFrame final creado con {df_final.shape[0]:,} filas y {df_final.shape[1]} columnas")
        print(f"\n📊 Columnas: {df_final.columns}")
        
        # Estadísticas
        print(f"\n{'='*80}")
        print("ESTADÍSTICAS")
        print(f"{'='*80}")
        
        if 'DPA_PARROQ' in df_final.columns:
            con_codigo = df_final.filter(pl.col('DPA_PARROQ').is_not_null()).shape[0]
            sin_codigo = df_final.filter(pl.col('DPA_PARROQ').is_null()).shape[0]
            print(f"  • Con código INEC: {con_codigo:,} ({con_codigo/df_final.shape[0]*100:.2f}%)")
            print(f"  • Sin código INEC: {sin_codigo:,} ({sin_codigo/df_final.shape[0]*100:.2f}%)")
        
        # Distribución por archivo
        print(f"\n{'='*80}")
        print("DISTRIBUCIÓN POR ARCHIVO")
        print(f"{'='*80}")
        
        dist_archivos = df_final.group_by("archivo_origen").agg(
            pl.count().alias("cantidad")
        ).sort("archivo_origen")
        
        print(dist_archivos)
        
        # Guardar archivo final
        output_file = "todos_georreferenciados_2021_2025.csv"
        print(f"\n💾 Guardando archivo: {output_file}")
        df_final.write_csv(output_file)
        
        tamano_final_mb = os.path.getsize(output_file) / (1024 * 1024)
        print(f"✅ Archivo guardado exitosamente ({tamano_final_mb:.2f} MB)")
        
        print(f"\n{'='*80}")
        print("✅ PROCESO COMPLETADO")
        print(f"{'='*80}")
        
        return df_final
        
    except Exception as e:
        print(f"\n❌ Error al concatenar: {e}")
        print(f"\n🔍 Información de debug:")
        for i, df in enumerate(dataframes_normalizados):
            print(f"  DataFrame {i+1}: {df.shape[0]} filas x {df.shape[1]} columnas")
            print(f"    Columnas: {df.columns}")
        return None

if __name__ == "__main__":
    df_unificado = concatenar_archivos_georreferenciados()





CONCATENANDO ARCHIVOS GEORREFERENCIADOS CON POLARS

✅ Se encontraron 5 archivos:
  • ../2022\eventos_2022_completo_georreferenciado.csv (830.65 MB)
  • ../2023\eventos_2023_completo_georreferenciado.csv (785.30 MB)
  • ../2024\eventos_2024_completo_georreferenciado.csv (748.36 MB)
  • ../2025\eventos_2025_completo_georreferenciado.csv (484.57 MB)
  • emergencias_2021_completo_georreferenciado.csv (879.63 MB)

[1/5] Procesando eventos_2022_completo_georreferenciado.csv...
  ✓ Leído: 3,628,972 filas, 12 columnas
  ✓ Procesado correctamente

[2/5] Procesando eventos_2023_completo_georreferenciado.csv...
  ✓ Leído: 3,446,124 filas, 12 columnas
  ✓ Procesado correctamente

[3/5] Procesando eventos_2024_completo_georreferenciado.csv...
  ✓ Leído: 3,291,743 filas, 12 columnas
  ✓ Procesado correctamente

[4/5] Procesando eventos_2025_completo_georreferenciado.csv...
  ✓ Leído: 2,144,983 filas, 12 columnas
  ✓ Procesado correctamente

[5/5] Procesando emergencias_2021_completo_georreferenciado

C:\Users\JPbau\AppData\Local\Temp\ipykernel_7000\3822312874.py:131: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("cantidad")


shape: (5, 2)
┌─────────────────────────────────┬──────────┐
│ archivo_origen                  ┆ cantidad │
│ ---                             ┆ ---      │
│ str                             ┆ u32      │
╞═════════════════════════════════╪══════════╡
│ emergencias_2021_completo_geor… ┆ 3805108  │
│ eventos_2022_completo_georrefe… ┆ 3628972  │
│ eventos_2023_completo_georrefe… ┆ 3446124  │
│ eventos_2024_completo_georrefe… ┆ 3291743  │
│ eventos_2025_completo_georrefe… ┆ 2144983  │
└─────────────────────────────────┴──────────┘

💾 Guardando archivo: todos_georreferenciados_2021_2025.csv
✅ Archivo guardado exitosamente (3689.81 MB)

✅ PROCESO COMPLETADO


In [5]:
df_unificado.head()

Fecha,provincia,Canton,Cod_Parroquia,Parroquia,Servicio,Subtipo,prov_norm,canton_norm,parr_norm,DPA_PARROQ,archivo_origen
str,str,str,i64,str,str,str,str,str,str,str,str
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""presencia policial""","""pichincha""","""quito""","""quito distrito metropolitano, …","""170150.0""","""eventos_2022_completo_georrefe…"
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""escandalo en espacio publico""","""pichincha""","""quito""","""quito distrito metropolitano, …","""170150.0""","""eventos_2022_completo_georrefe…"
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""servicios municipales""","""apoyo institucional""","""pichincha""","""quito""","""quito distrito metropolitano, …","""170150.0""","""eventos_2022_completo_georrefe…"
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""desaparecido""","""pichincha""","""quito""","""quito distrito metropolitano, …","""170150.0""","""eventos_2022_completo_georrefe…"
"""2022-04-01""","""pichincha""","""quito""",170176,"""pintag""","""seguridad ciudadana""","""patrullaje policial en el sect…","""pichincha""","""quito""","""pintag""","""170176.0""","""eventos_2022_completo_georrefe…"


In [6]:
df_unificado.describe()

statistic,Fecha,provincia,Canton,Cod_Parroquia,Parroquia,Servicio,Subtipo,prov_norm,canton_norm,parr_norm,DPA_PARROQ,archivo_origen
str,str,str,str,f64,str,str,str,str,str,str,str,str
"""count""","""16316930""","""16316930""","""16316930""",1.6316926e7,"""16316930""","""16314763""","""16316904""","""16316930""","""16316930""","""16316930""","""16316114""","""16316930"""
"""null_count""","""0""","""0""","""0""",4.0,"""0""","""2167""","""26""","""0""","""0""","""0""","""816""","""0"""
"""mean""",null,null,null,121235.859931,null,null,null,null,null,null,null,null
"""std""",null,null,null,56427.266675,null,null,null,null,null,null,null,null
"""min""","""2021-07-01""","""azuay""","""24 de mayo""",10150.0,"""10 de agosto""","""gestion de riesgos""","""abandono""","""azuay""","""24 de mayo""","""10 de agosto""","""100150.0""","""emergencias_2021_completo_geor…"
"""25%""",null,null,null,90150.0,null,null,null,null,null,null,null,null
"""50%""",null,null,null,110153.0,null,null,null,null,null,null,null,null
"""75%""",null,null,null,170150.0,null,null,null,null,null,null,null,null
"""max""","""2025-08-31""","""zamora chinchipe""","""zaruma""",240352.0,"""zurmi""","""transito y movilidad""","""zonal sin servicio de aseo""","""zamora chinchipe""","""zaruma""","""zurmi""","""92850.0""","""eventos_2025_completo_georrefe…"


In [7]:
type(df_unificado)


polars.dataframe.frame.DataFrame

In [8]:
df_copia = df_unificado.clone()
  

In [9]:
df_copia.head()

Fecha,provincia,Canton,Cod_Parroquia,Parroquia,Servicio,Subtipo,prov_norm,canton_norm,parr_norm,DPA_PARROQ,archivo_origen
str,str,str,i64,str,str,str,str,str,str,str,str
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""presencia policial""","""pichincha""","""quito""","""quito distrito metropolitano, …","""170150.0""","""eventos_2022_completo_georrefe…"
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""escandalo en espacio publico""","""pichincha""","""quito""","""quito distrito metropolitano, …","""170150.0""","""eventos_2022_completo_georrefe…"
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""servicios municipales""","""apoyo institucional""","""pichincha""","""quito""","""quito distrito metropolitano, …","""170150.0""","""eventos_2022_completo_georrefe…"
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""desaparecido""","""pichincha""","""quito""","""quito distrito metropolitano, …","""170150.0""","""eventos_2022_completo_georrefe…"
"""2022-04-01""","""pichincha""","""quito""",170176,"""pintag""","""seguridad ciudadana""","""patrullaje policial en el sect…","""pichincha""","""quito""","""pintag""","""170176.0""","""eventos_2022_completo_georrefe…"


In [17]:
df_copia.select(df_copia.filter(pl.col("DPA_PARROQ").is_null()))

Fecha,provincia,Canton,Cod_Parroquia,Parroquia,Servicio,Subtipo,prov_norm,canton_norm,parr_norm,DPA_PARROQ,archivo_origen
str,str,str,i64,str,str,str,str,str,str,str,str
"""2022-04-01""","""orellana""","""aguarico""",220254,"""tiputini""","""seguridad ciudadana""","""desaparicion de persona""","""orellana""","""aguarico""","""tiputini""",null,"""eventos_2022_completo_georrefe…"
"""2022-04-01""","""orellana""","""aguarico""",220254,"""tiputini""","""seguridad ciudadana""","""manifestaciones""","""orellana""","""aguarico""","""tiputini""",null,"""eventos_2022_completo_georrefe…"
"""2022-04-03""","""orellana""","""aguarico""",220254,"""tiputini""","""seguridad ciudadana""","""presencia policial""","""orellana""","""aguarico""","""tiputini""",null,"""eventos_2022_completo_georrefe…"
"""2022-04-03""","""orellana""","""aguarico""",220254,"""tiputini""","""seguridad ciudadana""","""robo personas""","""orellana""","""aguarico""","""tiputini""",null,"""eventos_2022_completo_georrefe…"
"""2022-04-03""","""orellana""","""aguarico""",220254,"""tiputini""","""seguridad ciudadana""","""presencia policial""","""orellana""","""aguarico""","""tiputini""",null,"""eventos_2022_completo_georrefe…"
…,…,…,…,…,…,…,…,…,…,…,…
"""2021-09-16""","""azuay""","""camilo ponce enriquez""",11551,"""el carmen de pijili""","""gestion sanitaria""","""consulta medica""","""azuay""","""camilo ponce enriquez""","""el carmen de pijili""",null,"""emergencias_2021_completo_geor…"
"""2021-09-26""","""azuay""","""camilo ponce enriquez""",11551,"""el carmen de pijili""","""seguridad ciudadana""","""libadores""","""azuay""","""camilo ponce enriquez""","""el carmen de pijili""",null,"""emergencias_2021_completo_geor…"
"""2021-09-29""","""azuay""","""camilo ponce enriquez""",11551,"""el carmen de pijili""","""gestion sanitaria""","""mal estado general""","""azuay""","""camilo ponce enriquez""","""el carmen de pijili""",null,"""emergencias_2021_completo_geor…"


In [18]:
df_copia = df_copia.drop_nulls(subset=['DPA_PARROQ'])

In [19]:
df_copia.head()

Fecha,provincia,Canton,Cod_Parroquia,Parroquia,Servicio,Subtipo,prov_norm,canton_norm,parr_norm,DPA_PARROQ,archivo_origen
str,str,str,i64,str,str,str,str,str,str,str,str
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""presencia policial""","""pichincha""","""quito""","""quito distrito metropolitano, …","""170150.0""","""eventos_2022_completo_georrefe…"
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""escandalo en espacio publico""","""pichincha""","""quito""","""quito distrito metropolitano, …","""170150.0""","""eventos_2022_completo_georrefe…"
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""servicios municipales""","""apoyo institucional""","""pichincha""","""quito""","""quito distrito metropolitano, …","""170150.0""","""eventos_2022_completo_georrefe…"
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""desaparecido""","""pichincha""","""quito""","""quito distrito metropolitano, …","""170150.0""","""eventos_2022_completo_georrefe…"
"""2022-04-01""","""pichincha""","""quito""",170176,"""pintag""","""seguridad ciudadana""","""patrullaje policial en el sect…","""pichincha""","""quito""","""pintag""","""170176.0""","""eventos_2022_completo_georrefe…"


In [21]:
df_unificado.shape

(16316930, 12)

In [20]:
df_copia.shape

(16316114, 12)

In [ ]:
df_copia = df_copia.select(pl.exclude("prov_norm","canton_norm","parr_norm", "archivo_origen"))


Fecha,provincia,Canton,Cod_Parroquia,Parroquia,Servicio,Subtipo,DPA_PARROQ
str,str,str,i64,str,str,str,str
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""presencia policial""","""170150.0"""
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""escandalo en espacio publico""","""170150.0"""
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""servicios municipales""","""apoyo institucional""","""170150.0"""
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""desaparecido""","""170150.0"""
"""2022-04-01""","""pichincha""","""quito""",170176,"""pintag""","""seguridad ciudadana""","""patrullaje policial en el sect…","""170176.0"""


In [23]:
df_copia

Fecha,provincia,Canton,Cod_Parroquia,Parroquia,Servicio,Subtipo,DPA_PARROQ
str,str,str,i64,str,str,str,str
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""presencia policial""","""170150.0"""
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""escandalo en espacio publico""","""170150.0"""
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""servicios municipales""","""apoyo institucional""","""170150.0"""
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""desaparecido""","""170150.0"""
"""2022-04-01""","""pichincha""","""quito""",170176,"""pintag""","""seguridad ciudadana""","""patrullaje policial en el sect…","""170176.0"""
…,…,…,…,…,…,…,…
"""2021-09-30""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""robo""","""170150.0"""
"""2021-09-30""","""pichincha""","""quito""",170160,"""el quinche""","""seguridad ciudadana""","""violencia contra la mujer o mi…","""170160.0"""
"""2021-09-30""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""presencia policial""","""170150.0"""


In [24]:
df_copia["DPA_PARROQ"]

DPA_PARROQ
str
"""170150.0"""
"""170150.0"""
"""170150.0"""
"""170150.0"""
"""170176.0"""
…
"""170150.0"""
"""170160.0"""
"""170150.0"""


In [26]:
# Verificar si TODOS los valores terminan en ".0"
todos_terminan_en_punto_cero = df_copia['DPA_PARROQ'].cast(pl.Utf8).str.ends_with('.0').all()
print(f"¿Todos terminan en .0?: {todos_terminan_en_punto_cero}")

¿Todos terminan en .0?: True


In [27]:
print (df_copia.select('DPA_PARROQ').head())
df_copia = df_copia.with_columns(
    pl.col('DPA_PARROQ').cast(pl.Utf8).str.strip_suffix(".0").alias('DPA_PARROQ')

)
print (df_copia.select('DPA_PARROQ').head())

shape: (5, 1)
┌────────────┐
│ DPA_PARROQ │
│ ---        │
│ str        │
╞════════════╡
│ 170150.0   │
│ 170150.0   │
│ 170150.0   │
│ 170150.0   │
│ 170176.0   │
└────────────┘
shape: (5, 1)
┌────────────┐
│ DPA_PARROQ │
│ ---        │
│ str        │
╞════════════╡
│ 170150     │
│ 170150     │
│ 170150     │
│ 170150     │
│ 170176     │
└────────────┘


In [30]:
df_copia

Fecha,provincia,Canton,Cod_Parroquia,Parroquia,Servicio,Subtipo,DPA_PARROQ
str,str,str,i64,str,str,str,str
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""presencia policial""","""170150"""
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""escandalo en espacio publico""","""170150"""
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""servicios municipales""","""apoyo institucional""","""170150"""
"""2022-04-01""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""desaparecido""","""170150"""
"""2022-04-01""","""pichincha""","""quito""",170176,"""pintag""","""seguridad ciudadana""","""patrullaje policial en el sect…","""170176"""
…,…,…,…,…,…,…,…
"""2021-09-30""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""robo""","""170150"""
"""2021-09-30""","""pichincha""","""quito""",170160,"""el quinche""","""seguridad ciudadana""","""violencia contra la mujer o mi…","""170160"""
"""2021-09-30""","""pichincha""","""quito""",170150,"""quito distrito metropolitano, …","""seguridad ciudadana""","""presencia policial""","""170150"""


In [32]:
df_copia.write_csv("datos_limpios_2021_2025.csv")